In [1]:
import numpy as np
import pandas as pd
import csv
import random
import scipy
import matplotlib
import matplotlib.pyplot as plt
from scipy import stats
import seaborn as sns
import statsmodels.api as sm
from statsmodels.formula.api import ols

In [2]:
"""Pareto smoothed importance sampling (PSIS)

This module implements Pareto smoothed importance sampling (PSIS) and PSIS
leave-one-out (LOO) cross-validation for Python (Numpy).

Included functions
------------------
psisloo
    Pareto smoothed importance sampling leave-one-out log predictive densities.

psislw
    Pareto smoothed importance sampling.

gpdfitnew
    Estimate the paramaters for the Generalized Pareto Distribution (GPD).

gpinv
    Inverse Generalised Pareto distribution function.

sumlogs
    Sum of vector where numbers are represented by their logarithms.

References
----------
Aki Vehtari, Andrew Gelman and Jonah Gabry (2017). Practical
Bayesian model evaluation using leave-one-out cross-validation
and WAIC. Statistics and Computing, 27(5):1413–1432.
doi:10.1007/s11222-016-9696-4. https://arxiv.org/abs/1507.04544

Aki Vehtari, Andrew Gelman and Jonah Gabry (2017). Pareto
smoothed importance sampling. https://arxiv.org/abs/arXiv:1507.02646v5

"""

from __future__ import division # For Python 2 compatibility
import numpy as np

# 3-Clause BSD License
"""
Copyright 2017 Aki Vehtari, Tuomas Sivula

Redistribution and use in source and binary forms, with or without modification,
are permitted provided that the following conditions are met:

1. Redistributions of source code must retain the above copyright notice, this
list of conditions and the following disclaimer.

2. Redistributions in binary form must reproduce the above copyright notice,
this list of conditions and the following disclaimer in the documentation and/or
other materials provided with the distribution.

3. Neither the name of the copyright holder nor the names of its contributors
may be used to endorse or promote products derived from this software without
specific prior written permission.

THIS SOFTWARE IS PROVIDED BY THE COPYRIGHT HOLDERS AND CONTRIBUTORS "AS IS" AND
ANY EXPRESS OR IMPLIED WARRANTIES, INCLUDING, BUT NOT LIMITED TO, THE IMPLIED
WARRANTIES OF MERCHANTABILITY AND FITNESS FOR A PARTICULAR PURPOSE ARE
DISCLAIMED. IN NO EVENT SHALL THE COPYRIGHT HOLDER OR CONTRIBUTORS BE LIABLE FOR
ANY DIRECT, INDIRECT, INCIDENTAL, SPECIAL, EXEMPLARY, OR CONSEQUENTIAL DAMAGES
(INCLUDING, BUT NOT LIMITED TO, PROCUREMENT OF SUBSTITUTE GOODS OR SERVICES;
LOSS OF USE, DATA, OR PROFITS; OR BUSINESS INTERRUPTION) HOWEVER CAUSED AND ON
ANY THEORY OF LIABILITY, WHETHER IN CONTRACT, STRICT LIABILITY, OR TORT
(INCLUDING NEGLIGENCE OR OTHERWISE) ARISING IN ANY WAY OUT OF THE USE OF THIS
SOFTWARE, EVEN IF ADVISED OF THE POSSIBILITY OF SUCH DAMAGE. """


def psisloo(log_lik, **kwargs):
    r"""PSIS leave-one-out log predictive densities.

    Computes the log predictive densities given posterior samples of the log
    likelihood terms :math:`p(y_i|\theta^s)` in input parameter `log_lik`.
    Returns a sum of the leave-one-out log predictive densities `loo`,
    individual leave-one-out log predictive density terms `loos` and an estimate
    of Pareto tail indeces `ks`. The estimates are unreliable if tail index
    ``k > 0.7`` (see more in the references listed in the module docstring).

    Additional keyword arguments are passed to the :meth:`psislw()` function
    (see the corresponding documentation).

    Parameters
    ----------
    log_lik : ndarray
        Array of size n x m containing n posterior samples of the log likelihood
        terms :math:`p(y_i|\theta^s)`.

    Returns
    -------
    loo : scalar
        sum of the leave-one-out log predictive densities

    loos : ndarray
        individual leave-one-out log predictive density terms

    ks : ndarray
        estimated Pareto tail indeces

    """
    # ensure overwrite flag in passed arguments
    kwargs['overwrite_lw'] = True
    # log raw weights from log_lik
    lw = -log_lik
    # compute Pareto smoothed log weights given raw log weights
    lw, ks = psislw(lw, **kwargs)
    # compute
    lw += log_lik
    loos = sumlogs(lw, axis=0)
    loo = loos.sum()
    return loo, loos, ks


def psislw(lw, Reff=1.0, overwrite_lw=False):
    """Pareto smoothed importance sampling (PSIS).

    Parameters
    ----------
    lw : ndarray
        Array of size n x m containing m sets of n log weights. It is also
        possible to provide one dimensional array of length n.

    Reff : scalar, optional
        relative MCMC efficiency ``N_eff / N``

    overwrite_lw : bool, optional
        If True, the input array `lw` is smoothed in-place, assuming the array
        is F-contiguous. By default, a new array is allocated.

    Returns
    -------
    lw_out : ndarray
        smoothed log weights
    kss : ndarray
        Pareto tail indices

    """
    if lw.ndim == 2:
        n, m = lw.shape
    elif lw.ndim == 1:
        n = len(lw)
        m = 1
    else:
        raise ValueError("Argument `lw` must be 1 or 2 dimensional.")
    if n <= 1:
        raise ValueError("More than one log-weight needed.")

    if overwrite_lw and lw.flags.f_contiguous:
        # in-place operation
        lw_out = lw
    else:
        # allocate new array for output
        lw_out = np.copy(lw, order='F')

    # allocate output array for kss
    kss = np.empty(m)

    # precalculate constants
    cutoff_ind = - int(np.ceil(min(0.2 * n, 3 * np.sqrt(n / Reff)))) - 1
    cutoffmin = np.log(np.finfo(float).tiny)
    logn = np.log(n)
    k_min = 1/3

    # loop over sets of log weights
    for i, x in enumerate(lw_out.T if lw_out.ndim == 2 else lw_out[None, :]):
        # improve numerical accuracy
        x -= np.max(x)
        # sort the array
        x_sort_ind = np.argsort(x)
        # divide log weights into body and right tail
        xcutoff = max(
            x[x_sort_ind[cutoff_ind]],
            cutoffmin
        )
        expxcutoff = np.exp(xcutoff)
        tailinds, = np.where(x > xcutoff)
        x2 = x[tailinds]
        n2 = len(x2)
        if n2 <= 4:
            # not enough tail samples for gpdfitnew
            k = np.inf
        else:
            # order of tail samples
            x2si = np.argsort(x2)
            # fit generalized Pareto distribution to the right tail samples
            np.exp(x2, out=x2)
            x2 -= expxcutoff
            k, sigma = gpdfitnew(x2, sort=x2si)
        if k >= k_min and not np.isinf(k):
            # no smoothing if short tail or GPD fit failed
            # compute ordered statistic for the fit
            sti = np.arange(0.5, n2)
            sti /= n2
            qq = gpinv(sti, k, sigma)
            qq += expxcutoff
            np.log(qq, out=qq)
            # place the smoothed tail into the output array
            x[tailinds[x2si]] = qq
            # truncate smoothed values to the largest raw weight 0
            x[x > 0] = 0
        # renormalize weights
        x -= sumlogs(x)
        # store tail index k
        kss[i] = k

    # If the provided input array is one dimensional, return kss as scalar.
    if lw_out.ndim == 1:
        kss = kss[0]

    return lw_out, kss


def gpdfitnew(x, sort=True, sort_in_place=False, return_quadrature=False):
    """Estimate the paramaters for the Generalized Pareto Distribution (GPD)

    Returns empirical Bayes estimate for the parameters of the two-parameter
    generalized Parato distribution given the data.

    Parameters
    ----------
    x : ndarray
        One dimensional data array

    sort : bool or ndarray, optional
        If known in advance, one can provide an array of indices that would
        sort the input array `x`. If the input array is already sorted, provide
        False. If True (default behaviour), the array is sorted internally.

    sort_in_place : bool, optional
        If `sort` is True and `sort_in_place` is True, the array is sorted
        in-place (False by default).

    return_quadrature : bool, optional
        If True, quadrature points and weight `ks` and `w` of the marginal posterior distribution of k are also calculated and returned. False by
        default.

    Returns
    -------
    k, sigma : float
        estimated parameter values

    ks, w : ndarray
        Quadrature points and weights of the marginal posterior distribution
        of `k`. Returned only if `return_quadrature` is True.

    Notes
    -----
    This function returns a negative of Zhang and Stephens's k, because it is
    more common parameterisation.

    """
    if x.ndim != 1 or len(x) <= 1:
        raise ValueError("Invalid input array.")

    # check if x should be sorted
    if sort is True:
        if sort_in_place:
            x.sort()
            xsorted = True
        else:
            sort = np.argsort(x)
            xsorted = False
    elif sort is False:
        xsorted = True
    else:
        xsorted = False

    n = len(x)
    PRIOR = 3
    m = 30 + int(np.sqrt(n))

    bs = np.arange(1, m + 1, dtype=float)
    bs -= 0.5
    np.divide(m, bs, out=bs)
    np.sqrt(bs, out=bs)
    np.subtract(1, bs, out=bs)
    if xsorted:
        bs /= PRIOR * x[int(n/4 + 0.5) - 1]
        bs += 1 / x[-1]
    else:
        bs /= PRIOR * x[sort[int(n/4 + 0.5) - 1]]
        bs += 1 / x[sort[-1]]

    ks = np.negative(bs)
    temp = ks[:,None] * x
    np.log1p(temp, out=temp)
    np.mean(temp, axis=1, out=ks)

    L = bs / ks
    np.negative(L, out=L)
    np.log(L, out=L)
    L -= ks
    L -= 1
    L *= n

    temp = L - L[:,None]
    np.exp(temp, out=temp)
    w = np.sum(temp, axis=1)
    np.divide(1, w, out=w)

    # remove negligible weights
    dii = w >= 10 * np.finfo(float).eps
    if not np.all(dii):
        w = w[dii]
        bs = bs[dii]
    # normalise w
    w /= w.sum()

    # posterior mean for b
    b = np.sum(bs * w)
    # Estimate for k, note that we return a negative of Zhang and
    # Stephens's k, because it is more common parameterisation.
    temp = (-b) * x
    np.log1p(temp, out=temp)
    k = np.mean(temp)
    if return_quadrature:
        np.negative(x, out=temp)
        temp = bs[:, None] * temp
        np.log1p(temp, out=temp)
        ks = np.mean(temp, axis=1)
    # estimate for sigma
    sigma = -k / b * n / (n - 0)
    # weakly informative prior for k
    a = 10
    k = k * n / (n+a) + a * 0.5 / (n+a)
    if return_quadrature:
        ks *= n / (n+a)
        ks += a * 0.5 / (n+a)

    if return_quadrature:
        return k, sigma, ks, w
    else:
        return k, sigma


def gpinv(p, k, sigma):
    """Inverse Generalised Pareto distribution function."""
    x = np.empty(p.shape)
    x.fill(np.nan)
    if sigma <= 0:
        return x
    ok = (p > 0) & (p < 1)
    if np.all(ok):
        if np.abs(k) < np.finfo(float).eps:
            np.negative(p, out=x)
            np.log1p(x, out=x)
            np.negative(x, out=x)
        else:
            np.negative(p, out=x)
            np.log1p(x, out=x)
            x *= -k
            np.expm1(x, out=x)
            x /= k
        x *= sigma
    else:
        if np.abs(k) < np.finfo(float).eps:
            # x[ok] = - np.log1p(-p[ok])
            temp = p[ok]
            np.negative(temp, out=temp)
            np.log1p(temp, out=temp)
            np.negative(temp, out=temp)
            x[ok] = temp
        else:
            # x[ok] = np.expm1(-k * np.log1p(-p[ok])) / k
            temp = p[ok]
            np.negative(temp, out=temp)
            np.log1p(temp, out=temp)
            temp *= -k
            np.expm1(temp, out=temp)
            temp /= k
            x[ok] = temp
        x *= sigma
        x[p == 0] = 0
        if k >= 0:
            x[p == 1] = np.inf
        else:
            x[p == 1] = -sigma / k
    return x


def sumlogs(x, axis=None, out=None):
    """Sum of vector where numbers are represented by their logarithms.

    Calculates ``np.log(np.sum(np.exp(x), axis=axis))`` in such a fashion that
    it works even when elements have large magnitude.

    """
    maxx = x.max(axis=axis, keepdims=True)
    xnorm = x - maxx
    np.exp(xnorm, out=xnorm)
    out = np.sum(xnorm, axis=axis, out=out)
    if isinstance(out, np.ndarray):
        np.log(out, out=out)
    else:
        out = np.log(out)
    out += np.squeeze(maxx)
    return out

In [3]:
subject_num = 60
def read_data(n):  #read behavioral data; no action: action[i]=4
    #数据文件在的位置
    action = []
    reward = []
    reward_B = []
    comparision = [] #r-r_b; 1:>=0, 0:<0
    
    subject = [1,2,3,4,5,6,7,8,9,10,
               11,12,13,14,15,16,17,18,19,20,
               21,22,23,24,25,26,27,28,29,30,
               31,32,33,34,35,36,37,38,39,40,
               41,42,43,44,45,46,47,48,49,50,
               51,52,53,54,55,56,58,59,60,61]
    
    num = str(100+subject[n])
    file = pd.read_csv('./data/' + num + '.csv')
    name = 'sub (' +str(n) +').csv'
    action_index = 0
    for i in  range((file.shape[1])):
        if file.columns[i] == 'choose_bandit.keys':
            action_index = i
    reward_index = 0
    for i in  range((file.shape[1])):
        if file.columns[i] == 'subchoose':
            reward_index = i
    for i in range((file.shape[0])):

        if file.iloc[i,0] == 1 and file.iloc[i,action_index]!='None' :
            if file.iloc[i,action_index] == 'r':
                action.append(0)
            elif file.iloc[i,action_index] == 'f':
                action.append(1)
            elif file.iloc[i,action_index] == 'i':
                action.append(2)
            elif file.iloc[i,action_index] == 'j':
                action.append(3)
            else:
                print(file.iloc[i,action_index])
                print(i)
                raise ValueError('不能识别选项')
            reward.append(int(file.iloc[i,reward_index]))
            reward_B.append(int(file.iloc[i,reward_index+1]))
            if int(file.iloc[i,reward_index]) >= int(file.iloc[i,reward_index+1]):
                comparision.append(int(file.iloc[i,reward_index])-int(file.iloc[i,reward_index+1]))
            else:
                comparision.append(int(file.iloc[i,reward_index+1])-int(file.iloc[i,reward_index]))
        elif file.iloc[i,0] == 1 and file.iloc[i,action_index] =='None' :
            action.append(4)
            reward.append(0)
            reward_B.append(0)
            comparision.append(0)
    return action,reward,reward_B,comparision

def read_data_all_reward():
    #read behavioral data; no action: action[i]=4
    #数据文件在的位置
    ACTION = []
    REWARD = []
    REWARD_B = []
    COMPARISON = [] #r-r_b; 1:>=0, 0:<0
    REWARD_ALL = []
    subject_num = 60
    subject = [1,2,3,4,5,6,7,8,9,10,
               11,12,13,14,15,16,17,18,19,20,
               21,22,23,24,25,26,27,28,29,30,
               31,32,33,34,35,36,37,38,39,40,
               41,42,43,44,45,46,47,48,49,50,
               51,52,53,54,55,56,58,59,60,61]
    for n in range(subject_num):
        action = []
        reward = []
        reward_B = []
        comparision = [] #r-r_b; 1:>=0, 0:<0
        reward_0 = []
        reward_1 = []
        reward_2 = []
        reward_3 = []
        num = str(100+subject[n])
        file = pd.read_csv('./data/' + num + '.csv')
        action_index = 0
        for i in  range((file.shape[1])):
            if file.columns[i] == 'choose_bandit.keys':
                action_index = i
        reward_index = 0
        for i in  range((file.shape[1])):
            if file.columns[i] == 'subchoose':
                reward_index = i
        for i in range((file.shape[0])):

            if file.iloc[i,0] == 1 and file.iloc[i,action_index]!='None' :
                if file.iloc[i,action_index] == 'r':
                    action.append(0)
                elif file.iloc[i,action_index] == 'f':
                    action.append(1)
                elif file.iloc[i,action_index] == 'i':
                    action.append(2)
                elif file.iloc[i,action_index] == 'j':
                    action.append(3)
                else:
                    print(file.iloc[i,action_index])
                    print(i)
                    raise ValueError('不能识别选项')
                reward.append(int(file.iloc[i,reward_index]))
                reward_B.append(int(file.iloc[i,reward_index+1]))
                reward_0.append(int(file.iloc[i,reward_index-4]))
                reward_1.append(int(file.iloc[i,reward_index-3]))
                reward_2.append(int(file.iloc[i,reward_index-2]))
                reward_3.append(int(file.iloc[i,reward_index-1]))
                # reward_all.append([int(file.iloc[i,reward_index-4]),int(file.iloc[i,reward_index-3]),int(file.iloc[i,reward_index-2]),int(file.iloc[i,reward_index-1])])
                if int(file.iloc[i,reward_index]) >= int(file.iloc[i,reward_index+1]):
                    comparision.append(1)
                else:
                    comparision.append(0)
            elif file.iloc[i,0] == 1 and file.iloc[i,action_index] =='None' :
                action.append(4)
                reward.append(0)
                reward_B.append(0)
                comparision.append(0)
                reward_0.append(int(file.iloc[i,reward_index-4]))
                reward_1.append(int(file.iloc[i,reward_index-3]))
                reward_2.append(int(file.iloc[i,reward_index-2]))
                reward_3.append(int(file.iloc[i,reward_index-1]))
                # reward_all.append([0,0,0,0])
        ACTION.append(action)
        REWARD.append(reward)
        REWARD_B.append(reward_B)
        COMPARISON.append(comparision)
        REWARD_ALL.append([reward_0,reward_1,reward_2,reward_3])
    return ACTION,REWARD,REWARD_B,COMPARISON,REWARD_ALL

def read_data_non_social():
    ACTION = []
    REWARD = []
    subject_num = 143
    for k in range(subject_num):
        action=[]
        reward=[]
        num = str(1+k)
        file = pd.read_csv('E:/multi-bandit/e1_new/bandit/extracted_data/PARTICIPANT_bandit_online_fixed_'+num+'.csv')
        trial_index = 0
        for i in  range((file.shape[1])):
            if file.columns[i] == 'trials.ran':
                trial_index = i
        action_index = 3
        for i in  range((file.shape[1])):
            if file.columns[i] == 'choose_bandit.keys':
                action_index = i
        reward_r_index = 7
        reward_f_index = 4
        reward_i_index = 5
        reward_j_index = 6
        for i in  range((file.shape[1])):
            if file.columns[i] == 'redpoints':
                reward_f_index = i
        for i in  range((file.shape[1])):
            if file.columns[i] == 'yellowpoints':
                reward_r_index = i
        for i in  range((file.shape[1])):
            if file.columns[i] == 'greenpoints':
                reward_j_index = i
        for i in  range((file.shape[1])):
            if file.columns[i] == 'bluepoints':
                reward_i_index = i
                
                
        for i in range((file.shape[0])):

            if file.iloc[i,trial_index] == 1.0 and(not pd.isna(file.iloc[i,action_index])) :
                if file.iloc[i,action_index] == 'r':
                    action.append(0)
                    reward.append(int(file.iloc[i,reward_r_index])) 
                elif file.iloc[i,action_index] == 'f':
                    action.append(1)
                    reward.append(int(file.iloc[i,reward_f_index])) 
                elif file.iloc[i,action_index] == 'i':
                    action.append(2)
                    reward.append(int(file.iloc[i,reward_i_index])) 
                elif file.iloc[i,action_index] == 'j':
                    action.append(3)
                    reward.append(int(file.iloc[i,reward_j_index])) 
                else:
                    print(file.iloc[i,action_index])
                    print(i)
                    raise ValueError('不能识别选项')
                               
            elif file.iloc[i,trial_index] == 1.0 and pd.isna(file.iloc[i,action_index]) :
                action.append(4)
                reward.append(0)
        ACTION.append(action)
        REWARD.append(reward)
    return np.array(ACTION),np.array(REWARD)

def read_data_all_reward_non_social():
    ACTION = []
    REWARD = []
    REWARD_ALL = []
    subject_num = 143
    for k in range(subject_num):
        action=[]
        reward=[]
        reward_0 = []
        reward_1 = []
        reward_2 = []
        reward_3 = []
        num = str(1+k)
        file = pd.read_csv('E:/multi-bandit/e1_new/bandit/extracted_data/PARTICIPANT_bandit_online_fixed_'+num+'.csv')
        trial_index = 0
        for i in  range((file.shape[1])):
            if file.columns[i] == 'trials.ran':
                trial_index = i
        action_index = 3
        for i in  range((file.shape[1])):
            if file.columns[i] == 'choose_bandit.keys':
                action_index = i
        reward_r_index = 7
        reward_f_index = 4
        reward_i_index = 5
        reward_j_index = 6
        for i in  range((file.shape[1])):
            if file.columns[i] == 'redpoints':
                reward_f_index = i
        for i in  range((file.shape[1])):
            if file.columns[i] == 'yellowpoints':
                reward_r_index = i
        for i in  range((file.shape[1])):
            if file.columns[i] == 'greenpoints':
                reward_j_index = i
        for i in  range((file.shape[1])):
            if file.columns[i] == 'bluepoints':
                reward_i_index = i
                
                
        for i in range((file.shape[0])):

            if file.iloc[i,trial_index] == 1.0 and(not pd.isna(file.iloc[i,action_index])) :
                # print(file.iloc[i,action_index])
                # print(file.iloc[i,12])
                if file.iloc[i,action_index] == 'r':
                    action.append(0)
                    reward.append(int(file.iloc[i,reward_r_index])) 
                    
                elif file.iloc[i,action_index] == 'f':
                    action.append(1)
                    reward.append(int(file.iloc[i,reward_f_index])) 
                    
                elif file.iloc[i,action_index] == 'i':
                    action.append(2)
                    reward.append(int(file.iloc[i,reward_i_index])) 
                    
                elif file.iloc[i,action_index] == 'j':
                    action.append(3)
                    reward.append(int(file.iloc[i,reward_j_index])) 
                    
                else:
                    print(file.iloc[i,action_index])
                    print(i)
                    raise ValueError('不能识别选项')
                reward_0.append(int(file.iloc[i,reward_r_index]))   
                reward_1.append(int(file.iloc[i,reward_f_index])) 
                reward_2.append(int(file.iloc[i,reward_i_index]))
                reward_3.append(int(file.iloc[i,reward_j_index]))
            elif file.iloc[i,trial_index] == 1.0 and pd.isna(file.iloc[i,action_index]) :
                action.append(4)
                reward.append(0)
                reward_0.append(int(file.iloc[i,reward_r_index]))
                reward_1.append(int(file.iloc[i,reward_f_index]))
                reward_2.append(int(file.iloc[i,reward_i_index]))
                reward_3.append(int(file.iloc[i,reward_j_index]))
        # print(len(action))
        ACTION.append(action)
        REWARD.append(reward)
        REWARD_ALL.append([reward_0,reward_1,reward_2,reward_3])
    return np.array(ACTION),np.array(REWARD),np.array(REWARD_ALL)
    

def read_data_happiness(n):  #read behavioral data; no action: action[i]=4
    #数据文件在的位置
    action = []
    reward = []
    reward_B = []
    happiness = [] #r-r_b; 1:>=0, 0:<0
    subject = [1,2,3,4,5,6,7,8,9,10,
               11,12,13,14,15,16,17,18,19,20,
               21,22,23,24,25,26,27,28,29,30,
               31,32,33,34,35,36,37,38,39,40,
               41,42,43,44,45,46,47,48,49,50,
               51,52,53,54,55,56,58,59,60,61]
    num = str(100+subject[n])
    file = pd.read_csv('./data/' + num + '.csv')
    action_index = 0
    for i in  range((file.shape[1])):
        if file.columns[i] == 'choose_bandit.keys':
            action_index = i
    reward_index = 0
    for i in  range((file.shape[1])):
        if file.columns[i] == 'subchoose':
            reward_index = i
    happiness_index = 0
    for i in range((file.shape[1])):
        if file.columns[i] == 'slider.response':
            happiness_index = i
    
    for i in range((file.shape[0])):
        if file.iloc[i,0] == 1 and file.iloc[i,action_index]!='None' :
            if file.iloc[i,action_index] == 'r':
                action.append(0)
            elif file.iloc[i,action_index] == 'f':
                action.append(1)
            elif file.iloc[i,action_index] == 'i':
                action.append(2)
            elif file.iloc[i,action_index] == 'j':
                action.append(3)
            else:
                print(file.iloc[i,action_index])
                print(i)
                raise ValueError('不能识别选项')
            reward.append(int(file.iloc[i,reward_index]))
            reward_B.append(int(file.iloc[i,reward_index+1]))
            happiness.append(int(float(file.iloc[i,happiness_index])))
    return action,reward,reward_B,happiness

In [4]:
def read_parameter(file_name):
    parameter_result = pd.read_csv(file_name)
    beta = []
    phi = []
    persev = []
    gamma = []
    for i in range(10):
        beta.append(parameter_result.iloc[i,1])
    for i in range(10):
        phi.append(parameter_result.iloc[i+10,1])
    for i in range(10):
        persev.append(parameter_result.iloc[i+20,1])
    for i in range(10):
        gamma.append(parameter_result.iloc[i+30,1])
    beta = np.array(beta)
    phi = np.array(phi)
    persev = np.array(persev)
    gamma = np.array(gamma)
    return beta,phi,persev,gamma

def action_probability(Q,action,beta):
    return np.log(np.exp(Q[action]*beta)/(np.exp(Q[0]*beta)+np.exp(Q[1]*beta)+np.exp(Q[2]*beta)+np.exp(Q[3]*beta)))
    

In [18]:
def read_effective_num_9(file_name):
    parameter_result = pd.read_csv(file_name)
    subject_num = 60
    para1 = []
    para2 = []
    para3 = []
    para4 = []
    para5 = []
    para6 = []
    para7 = []
    para8 = []
    para9 = []
    for i in range(subject_num):
        para1.append(parameter_result.iloc[i,9])
    for i in range(subject_num):
        para2.append(parameter_result.iloc[i+subject_num,9])
    for i in range(subject_num):
        para3.append(parameter_result.iloc[i+subject_num*2,9])
    for i in range(subject_num):
        para4.append(parameter_result.iloc[i+subject_num*3,9])
    for i in range(subject_num):
        para5.append(parameter_result.iloc[i+subject_num*4,9])
    for i in range(subject_num):
        para6.append(parameter_result.iloc[i+subject_num*5,9])
    for i in range(subject_num):
        para7.append(parameter_result.iloc[i+subject_num*6,9])
    for i in range(subject_num):
        para8.append(parameter_result.iloc[i+subject_num*7,9])
    for i in range(subject_num):
        para9.append(parameter_result.iloc[i+subject_num*8,9])
    para1 = np.array(para1)
    para2 = np.array(para2)
    para3 = np.array(para3)
    para4 = np.array(para4)
    para5 = np.array(para5)
    para6 = np.array(para6)
    para7 = np.array(para7)
    para8 = np.array(para8)
    para9 = np.array(para9)
    return para1,para2,para3,para4,para5,para6,para7,para8,para9

def read_effective_num_6(file_name):
    parameter_result = pd.read_csv(file_name)
    subject_num = 60
    para1 = []
    para2 = []
    para3 = []
    para4 = []
    para5 = []
    para6 = []
    for i in range(subject_num):
        para1.append(parameter_result.iloc[i,9])
    for i in range(subject_num):
        para2.append(parameter_result.iloc[i+subject_num,9])
    for i in range(subject_num):
        para3.append(parameter_result.iloc[i+subject_num*2,9])
    for i in range(subject_num):
        para4.append(parameter_result.iloc[i+subject_num*3,9])
    for i in range(subject_num):
        para5.append(parameter_result.iloc[i+subject_num*4,9])
    for i in range(subject_num):
        para6.append(parameter_result.iloc[i+subject_num*5,9])
    para1 = np.array(para1)
    para2 = np.array(para2)
    para3 = np.array(para3)
    para4 = np.array(para4)
    para5 = np.array(para5)
    para6 = np.array(para6)
    return para1,para2,para3,para4,para5,para6

def read_effective_num_5(file_name):
    parameter_result = pd.read_csv(file_name)
    subject_num = 60
    para1 = []
    para2 = []
    para3 = []
    para4 = []
    para5 = []

    for i in range(subject_num):
        para1.append(parameter_result.iloc[i,9])
    for i in range(subject_num):
        para2.append(parameter_result.iloc[i+subject_num,9])
    for i in range(subject_num):
        para3.append(parameter_result.iloc[i+subject_num*2,9])
    for i in range(subject_num):
        para4.append(parameter_result.iloc[i+subject_num*3,9])
    for i in range(subject_num):
        para5.append(parameter_result.iloc[i+subject_num*4,9])

    para1 = np.array(para1)
    para2 = np.array(para2)
    para3 = np.array(para3)
    para4 = np.array(para4)
    para5 = np.array(para5)

    return para1,para2,para3,para4,para5

def read_effective_num_4(file_name):
    parameter_result = pd.read_csv(file_name)
    subject_num = 60
    para1 = []
    para2 = []
    para3 = []
    para4 = []

    for i in range(subject_num):
        para1.append(parameter_result.iloc[i,9])
    for i in range(subject_num):
        para2.append(parameter_result.iloc[i+subject_num,9])
    for i in range(subject_num):
        para3.append(parameter_result.iloc[i+subject_num*2,9])
    for i in range(subject_num):
        para4.append(parameter_result.iloc[i+subject_num*3,9])

    para1 = np.array(para1)
    para2 = np.array(para2)
    para3 = np.array(para3)
    para4 = np.array(para4)
    return para1,para2,para3,para4

def read_effective_num_4(file_name):
    parameter_result = pd.read_csv(file_name)
    subject_num = 60
    para1 = []
    para2 = []
    para3 = []
    para4 = []

    for i in range(subject_num):
        para1.append(parameter_result.iloc[i,9])
    for i in range(subject_num):
        para2.append(parameter_result.iloc[i+subject_num,9])
    for i in range(subject_num):
        para3.append(parameter_result.iloc[i+subject_num*2,9])
    for i in range(subject_num):
        para4.append(parameter_result.iloc[i+subject_num*3,9])

    para1 = np.array(para1)
    para2 = np.array(para2)
    para3 = np.array(para3)
    para4 = np.array(para4)
    return para1,para2,para3,para4

def read_effective_num_3(file_name):
    parameter_result = pd.read_csv(file_name)
    subject_num = 60
    para1 = []
    para2 = []
    para3 = []

    for i in range(subject_num):
        para1.append(parameter_result.iloc[i,9])
    for i in range(subject_num):
        para2.append(parameter_result.iloc[i+subject_num,9])
    for i in range(subject_num):
        para3.append(parameter_result.iloc[i+subject_num*2,9])

    para1 = np.array(para1)
    para2 = np.array(para2)
    para3 = np.array(para3)

    return para1,para2,para3

def read_effective_num_2(file_name):
    parameter_result = pd.read_csv(file_name)
    subject_num = 60
    para1 = []
    para2 = []

    for i in range(subject_num):
        para1.append(parameter_result.iloc[i,9])
    for i in range(subject_num):
        para2.append(parameter_result.iloc[i+subject_num,9])

    para1 = np.array(para1)
    para2 = np.array(para2)
    return para1,para2

def read_effective_num_1(file_name):
    parameter_result = pd.read_csv(file_name)
    subject_num = 60
    para1 = []

    for i in range(subject_num):
        para1.append(parameter_result.iloc[i,9])

    para1 = np.array(para1)
    return para1

def read_parameter_3_para(file_name):
    parameter_result = pd.read_csv(file_name)
    subject_num = 60
    para1 = []
    para2 = []
    para3 = []
    for i in range(subject_num):
        para1.append(parameter_result.iloc[i,1])
    for i in range(subject_num):
        para2.append(parameter_result.iloc[i+subject_num,1])
    for i in range(subject_num):
        para3.append(parameter_result.iloc[i+subject_num*2,1])
    para1 = np.array(para1)
    para2 = np.array(para2)
    para3 = np.array(para3)
    return para1,para2,para3
    
def read_parameter(file_name): # read fitted parameters: beta,phi,persev,gamma
    parameter_result = pd.read_csv(file_name)
    subject_num = 60
    beta = []
    phi = []
    persev = []
    gamma = []
    for i in range(subject_num):
        beta.append(parameter_result.iloc[i,1])
    for i in range(subject_num):
        phi.append(parameter_result.iloc[i+subject_num,1])
    for i in range(subject_num):
        persev.append(parameter_result.iloc[i+subject_num*2,1])
    for i in range(subject_num):
        gamma.append(parameter_result.iloc[i+subject_num*3,1])
    beta = np.array(beta)
    phi = np.array(phi)
    persev = np.array(persev)
    gamma = np.array(gamma)
    return beta,phi,persev,gamma

def read_parameter_sup(file_name): # read fitted parameters: beta,phi,persev,gamma
    parameter_result = pd.read_csv(file_name)
    subject_num = 60
    beta = []
    phi = []
    persev = []
    gamma = []
    for i in range(subject_num):
        beta.append(parameter_result.iloc[i,1])
    for i in range(subject_num):
        phi.append(parameter_result.iloc[i+subject_num,1])
    for i in range(subject_num):
        persev.append(parameter_result.iloc[i+subject_num*2,1])
    for i in range(subject_num):
        gamma.append(parameter_result.iloc[i+subject_num*3,1])
    beta = np.array(beta)
    phi = np.array(phi)
    persev = np.array(persev)
    gamma = np.array(gamma)
    return beta,phi,persev,gamma

def read_parameter_sup_sample(file_name): # read fitted parameters: beta,phi,persev,gamma
    parameter_result = pd.read_csv(file_name)
    subject_num = 60
    beta = []
    phi = []
    persev = []
    gamma = []
    
    beta_sd = []
    phi_sd = []
    persev_sd = []
    gamma_sd = []
    
    for i in range(subject_num):
        beta.append(parameter_result.iloc[i,1])
    for i in range(subject_num):
        phi.append(parameter_result.iloc[i+subject_num,1])
    for i in range(subject_num):
        persev.append(parameter_result.iloc[i+subject_num*2,1])
    for i in range(subject_num):
        gamma.append(parameter_result.iloc[i+subject_num*3,1])
        
    for i in range(subject_num):
        beta_sd.append(parameter_result.iloc[i,3])
    for i in range(subject_num):
        phi_sd.append(parameter_result.iloc[i+subject_num,3])
    for i in range(subject_num):
        persev_sd.append(parameter_result.iloc[i+subject_num*2,3])
    for i in range(subject_num):
        gamma_sd.append(parameter_result.iloc[i+subject_num*3,3])
    
    beta = np.array(beta)
    phi = np.array(phi)
    persev = np.array(persev)
    gamma = np.array(gamma)
    
    beta_sd = np.array(beta_sd)
    phi_sd = np.array(phi_sd)
    persev_sd = np.array(persev_sd)
    gamma_sd = np.array(gamma_sd)
    
    return beta,phi,persev,gamma,beta_sd,phi_sd,persev_sd,gamma_sd

def read_parameter_persev_trace_vep(file_name): # read fitted parameters: beta,phi,persev,gamma
    parameter_result = pd.read_csv(file_name)
    subject_num = 60
    beta_a = []
    phi_a = []
    persev_a = []
    gamma_a = []

    for i in range(subject_num):
        beta_a.append(parameter_result.iloc[i,1])
    for i in range(subject_num):
        phi_a.append(parameter_result.iloc[i+subject_num,1])
    for i in range(subject_num):
        persev_a.append(parameter_result.iloc[i+subject_num*2,1])
    for i in range(subject_num):
        gamma_a.append(parameter_result.iloc[i+subject_num*3,1])

    beta_a = np.array(beta_a)
    phi_a = np.array(phi_a)
    persev_a = np.array(persev_a)
    gamma_a = np.array(gamma_a)

    return beta_a,phi_a,persev_a,gamma_a

def read_parameter_persev_trace_vep_sample(file_name): # read fitted parameters: beta,phi,persev,gamma
    parameter_result = pd.read_csv(file_name)
    subject_num = 60
    beta_a = []
    phi_a = []
    persev_a = []
    gamma_a = []

    
    beta_a_sd = []
    phi_a_sd = []
    persev_a_sd = []
    gamma_a_sd = []


    for i in range(subject_num):
        beta_a.append(parameter_result.iloc[i,1])
    for i in range(subject_num):
        phi_a.append(parameter_result.iloc[i+subject_num,1])
    for i in range(subject_num):
        persev_a.append(parameter_result.iloc[i+subject_num*2,1])
    for i in range(subject_num):
        gamma_a.append(parameter_result.iloc[i+subject_num*3,1])

        
    for i in range(subject_num):
        beta_a_sd.append(parameter_result.iloc[i,3])
    for i in range(subject_num):
        phi_a_sd.append(parameter_result.iloc[i+subject_num,3])
    for i in range(subject_num):
        persev_a_sd.append(parameter_result.iloc[i+subject_num*2,3])
    for i in range(subject_num):
        gamma_a_sd.append(parameter_result.iloc[i+subject_num*3,3])


    beta_a = np.array(beta_a)
    phi_a = np.array(phi_a)
    persev_a = np.array(persev_a)
    gamma_a = np.array(gamma_a)
    
    beta_a_sd = np.array(beta_a_sd)
    phi_a_sd = np.array(phi_a_sd)
    persev_a_sd = np.array(persev_a_sd)
    gamma_a_sd = np.array(gamma_a_sd)

    return beta_a,phi_a,persev_a,gamma_a,beta_a_sd,phi_a_sd,persev_a_sd,gamma_a_sd

def read_parameter_persev_trace_glm_p(file_name): # read fitted parameters: beta,phi,persev,gamma
    parameter_result = pd.read_csv(file_name)
    subject_num = 60
    beta_a = []
    phi_a = []
    persev_a = []
    gamma_a = []
    beta_b = []


    for i in range(subject_num):
        beta_a.append(parameter_result.iloc[i,1])
    for i in range(subject_num):
        phi_a.append(parameter_result.iloc[i+subject_num,1])
    for i in range(subject_num):
        persev_a.append(parameter_result.iloc[i+subject_num*2,1])
    for i in range(subject_num):
        gamma_a.append(parameter_result.iloc[i+subject_num*3,1])
    for i in range(subject_num):
        beta_b.append(parameter_result.iloc[i+subject_num*4,1])


    beta_a = np.array(beta_a)
    phi_a = np.array(phi_a)
    persev_a = np.array(persev_a)
    gamma_a = np.array(gamma_a)
    beta_b = np.array(beta_b)


    return beta_a,phi_a,persev_a,gamma_a,beta_b

def read_parameter_persev_trace_glm_p_sample(file_name): # read fitted parameters: beta,phi,persev,gamma
    parameter_result = pd.read_csv(file_name)
    subject_num = 60
    beta_a = []
    phi_a = []
    persev_a = []
    gamma_a = []
    beta_b = []
    
    beta_a_sd = []
    phi_a_sd = []
    persev_a_sd = []
    gamma_a_sd = []
    beta_b_sd = []

    for i in range(subject_num):
        beta_a.append(parameter_result.iloc[i,1])
    for i in range(subject_num):
        phi_a.append(parameter_result.iloc[i+subject_num,1])
    for i in range(subject_num):
        persev_a.append(parameter_result.iloc[i+subject_num*2,1])
    for i in range(subject_num):
        gamma_a.append(parameter_result.iloc[i+subject_num*3,1])
    for i in range(subject_num):
        beta_b.append(parameter_result.iloc[i+subject_num*4,1])
        
    for i in range(subject_num):
        beta_a_sd.append(parameter_result.iloc[i,3])
    for i in range(subject_num):
        phi_a_sd.append(parameter_result.iloc[i+subject_num,3])
    for i in range(subject_num):
        persev_a_sd.append(parameter_result.iloc[i+subject_num*2,3])
    for i in range(subject_num):
        gamma_a_sd.append(parameter_result.iloc[i+subject_num*3,3])
    for i in range(subject_num):
        beta_b_sd.append(parameter_result.iloc[i+subject_num*4,3])

    beta_a = np.array(beta_a)
    phi_a = np.array(phi_a)
    persev_a = np.array(persev_a)
    gamma_a = np.array(gamma_a)
    beta_b = np.array(beta_b)
    
    beta_a_sd = np.array(beta_a_sd)
    phi_a_sd = np.array(phi_a_sd)
    persev_a_sd = np.array(persev_a_sd)
    gamma_a_sd = np.array(gamma_a_sd)
    beta_b_sd = np.array(beta_b_sd)

    return beta_a,phi_a,persev_a,gamma_a,beta_b,beta_a_sd,phi_a_sd,persev_a_sd,gamma_a_sd,beta_b_sd


def read_parameter_persev_trace(file_name): # read fitted parameters: beta,phi,persev,gamma
    parameter_result = pd.read_csv(file_name)
    subject_num = 60
    beta_a = []
    phi_a = []
    persev_a = []
    gamma_a = []
    beta_b = []
    phi_b = []
    persev_b = []

    for i in range(subject_num):
        beta_a.append(parameter_result.iloc[i,1])
    for i in range(subject_num):
        phi_a.append(parameter_result.iloc[i+subject_num,1])
    for i in range(subject_num):
        persev_a.append(parameter_result.iloc[i+subject_num*2,1])
    for i in range(subject_num):
        gamma_a.append(parameter_result.iloc[i+subject_num*3,1])
    for i in range(subject_num):
        beta_b.append(parameter_result.iloc[i+subject_num*4,1])
    for i in range(subject_num):
        phi_b.append(parameter_result.iloc[i+subject_num*5,1])
    for i in range(subject_num):
        persev_b.append(parameter_result.iloc[i+subject_num*6,1])

    beta_a = np.array(beta_a)
    phi_a = np.array(phi_a)
    persev_a = np.array(persev_a)
    gamma_a = np.array(gamma_a)
    beta_b = np.array(beta_b)
    phi_b = np.array(phi_b)
    persev_b = np.array(persev_b)

    return beta_a,phi_a,persev_a,gamma_a,beta_b,phi_b,persev_b

def read_parameter_persev_trace_sample(file_name): # read fitted parameters: beta,phi,persev,gamma
    parameter_result = pd.read_csv(file_name)
    subject_num = 60
    beta_a = []
    phi_a = []
    persev_a = []
    gamma_a = []
    beta_b = []
    phi_b = []
    persev_b = []
    
    beta_a_sd = []
    phi_a_sd = []
    persev_a_sd = []
    gamma_a_sd = []
    beta_b_sd = []
    phi_b_sd = []
    persev_b_sd = []

    for i in range(subject_num):
        beta_a.append(parameter_result.iloc[i,1])
    for i in range(subject_num):
        phi_a.append(parameter_result.iloc[i+subject_num,1])
    for i in range(subject_num):
        persev_a.append(parameter_result.iloc[i+subject_num*2,1])
    for i in range(subject_num):
        gamma_a.append(parameter_result.iloc[i+subject_num*3,1])
    for i in range(subject_num):
        beta_b.append(parameter_result.iloc[i+subject_num*4,1])
    for i in range(subject_num):
        phi_b.append(parameter_result.iloc[i+subject_num*5,1])
    for i in range(subject_num):
        persev_b.append(parameter_result.iloc[i+subject_num*6,1])
        
    for i in range(subject_num):
        beta_a_sd.append(parameter_result.iloc[i,3])
    for i in range(subject_num):
        phi_a_sd.append(parameter_result.iloc[i+subject_num,3])
    for i in range(subject_num):
        persev_a_sd.append(parameter_result.iloc[i+subject_num*2,3])
    for i in range(subject_num):
        gamma_a_sd.append(parameter_result.iloc[i+subject_num*3,3])
    for i in range(subject_num):
        beta_b_sd.append(parameter_result.iloc[i+subject_num*4,3])
    for i in range(subject_num):
        phi_b_sd.append(parameter_result.iloc[i+subject_num*5,3])
    for i in range(subject_num):
        persev_b_sd.append(parameter_result.iloc[i+subject_num*6,3])

    beta_a = np.array(beta_a)
    phi_a = np.array(phi_a)
    persev_a = np.array(persev_a)
    gamma_a = np.array(gamma_a)
    beta_b = np.array(beta_b)
    phi_b = np.array(phi_b)
    persev_b = np.array(persev_b)
    
    beta_a_sd = np.array(beta_a_sd)
    phi_a_sd = np.array(phi_a_sd)
    persev_a_sd = np.array(persev_a_sd)
    gamma_a_sd = np.array(gamma_a_sd)
    beta_b_sd = np.array(beta_b_sd)
    phi_b_sd = np.array(phi_b_sd)
    persev_b_sd = np.array(persev_b_sd)

    return beta_a,phi_a,persev_a,gamma_a,beta_b,phi_b,persev_b,beta_a_sd,phi_a_sd,persev_a_sd,gamma_a_sd,beta_b_sd,phi_b_sd,persev_b_sd


def read_parameter_sup_dual(file_name): # read fitted parameters: beta,phi,persev,gamma
    parameter_result = pd.read_csv(file_name)
    subject_num = 60
    beta = []
    phi = []
    persev = []
    gamma = []
    a = []
    for i in range(subject_num):
        beta.append(parameter_result.iloc[i,1])
    for i in range(subject_num):
        phi.append(parameter_result.iloc[i+subject_num,1])
    for i in range(subject_num):
        persev.append(parameter_result.iloc[i+subject_num*2,1])
    for i in range(subject_num):
        gamma.append(parameter_result.iloc[i+subject_num*3,1])
    for i in range(subject_num):
        a.append(parameter_result.iloc[i+subject_num*4,1])
    beta = np.array(beta)
    phi = np.array(phi)
    persev = np.array(persev)
    gamma = np.array(gamma)
    a = np.array(a)
    return beta,phi,persev,gamma,a

def read_parameter_sup_dual_sample(file_name): # read fitted parameters: beta,phi,persev,gamma
    parameter_result = pd.read_csv(file_name)
    subject_num = 60
    beta = []
    phi = []
    persev = []
    gamma = []
    a = []
    
    beta_sd = []
    phi_sd = []
    persev_sd = []
    gamma_sd = []
    a_sd = []
    
    for i in range(subject_num):
        beta.append(parameter_result.iloc[i,1])
    for i in range(subject_num):
        phi.append(parameter_result.iloc[i+subject_num,1])
    for i in range(subject_num):
        persev.append(parameter_result.iloc[i+subject_num*2,1])
    for i in range(subject_num):
        gamma.append(parameter_result.iloc[i+subject_num*3,1])
    for i in range(subject_num):
        a.append(parameter_result.iloc[i+subject_num*4,1])
        
    for i in range(subject_num):
        beta_sd.append(parameter_result.iloc[i,3])
    for i in range(subject_num):
        phi_sd.append(parameter_result.iloc[i+subject_num,3])
    for i in range(subject_num):
        persev_sd.append(parameter_result.iloc[i+subject_num*2,3])
    for i in range(subject_num):
        gamma_sd.append(parameter_result.iloc[i+subject_num*3,3])
    for i in range(subject_num):
        a_sd.append(parameter_result.iloc[i+subject_num*4,3])
    beta = np.array(beta)
    phi = np.array(phi)
    persev = np.array(persev)
    gamma = np.array(gamma)
    a = np.array(a)
    
    beta_sd = np.array(beta_sd)
    phi_sd = np.array(phi_sd)
    persev_sd = np.array(persev_sd)
    gamma_sd = np.array(gamma_sd)
    a_sd = np.array(a_sd)
    return beta,phi,persev,gamma,a,beta_sd,phi_sd,persev_sd,gamma_sd,a_sd

def read_parameter_glm_para(file_name): # read fitted parameters: beta,phi,persev,gamma
    parameter_result = pd.read_csv(file_name)
    subject_num = 60
    beta = []
    phi = []
    persev = []
    gamma = []
    glm_a = []
    for i in range(subject_num):
        beta.append(parameter_result.iloc[i,1])
    for i in range(subject_num):
        phi.append(parameter_result.iloc[i+subject_num,1])
    for i in range(subject_num):
        persev.append(parameter_result.iloc[i+subject_num*2,1])
    for i in range(subject_num):
        gamma.append(parameter_result.iloc[i+subject_num*3,1])
    for i in range(subject_num):
        glm_a.append(parameter_result.iloc[i+subject_num*4,1])
    beta = np.array(beta)
    phi = np.array(phi)
    persev = np.array(persev)
    gamma = np.array(gamma)
    glm_a = np.array(glm_a)
    return beta,phi,persev,gamma,glm_a

def read_parameter_glm_3_para(file_name): # read fitted parameters: beta_a,b;phi_a,b;persev_,b;gamma_a,b
    parameter_result = pd.read_csv(file_name)
    subject_num = 60
    beta_a = []
    phi_a = []
    persev_a = []

    beta_b = []
    phi_b = []
    persev_b = []

    for i in range(subject_num):
        beta_a.append(parameter_result.iloc[i,1])
    for i in range(subject_num):
        phi_a.append(parameter_result.iloc[i+subject_num,1])
    for i in range(subject_num):
        persev_a.append(parameter_result.iloc[i+subject_num*2,1])

    for i in range(subject_num):
        beta_b.append(parameter_result.iloc[i+subject_num*3,1])
    for i in range(subject_num):
        phi_b.append(parameter_result.iloc[i+subject_num*4,1])
    for i in range(subject_num):
        persev_b.append(parameter_result.iloc[i+subject_num*5,1])

    beta_a = np.array(beta_a)
    phi_a = np.array(phi_a)
    persev_a = np.array(persev_a)

    beta_b = np.array(beta_b)
    phi_b = np.array(phi_b)
    persev_b = np.array(persev_b)

    return beta_a,phi_a,persev_a,beta_b,phi_b,persev_b

def read_parameter_glm_3_para_sample(file_name): # read fitted parameters: beta_a,b;phi_a,b;persev_,b;gamma_a,b
    parameter_result = pd.read_csv(file_name)
    subject_num = 60
    beta_a = []
    phi_a = []
    persev_a = []

    beta_b = []
    phi_b = []
    persev_b = []
    
    beta_a_sd = []
    phi_a_sd = []
    persev_a_sd = []

    beta_b_sd = []
    phi_b_sd = []
    persev_b_sd = []

    for i in range(subject_num):
        beta_a.append(parameter_result.iloc[i,1])
    for i in range(subject_num):
        phi_a.append(parameter_result.iloc[i+subject_num,1])
    for i in range(subject_num):
        persev_a.append(parameter_result.iloc[i+subject_num*2,1])

    for i in range(subject_num):
        beta_b.append(parameter_result.iloc[i+subject_num*3,1])
    for i in range(subject_num):
        phi_b.append(parameter_result.iloc[i+subject_num*4,1])
    for i in range(subject_num):
        persev_b.append(parameter_result.iloc[i+subject_num*5,1])
        
    for i in range(subject_num):
        beta_a_sd.append(parameter_result.iloc[i,3])
    for i in range(subject_num):
        phi_a_sd.append(parameter_result.iloc[i+subject_num,3])
    for i in range(subject_num):
        persev_a_sd.append(parameter_result.iloc[i+subject_num*2,3])

    for i in range(subject_num):
        beta_b_sd.append(parameter_result.iloc[i+subject_num*3,3])
    for i in range(subject_num):
        phi_b_sd.append(parameter_result.iloc[i+subject_num*4,3])
    for i in range(subject_num):
        persev_b_sd.append(parameter_result.iloc[i+subject_num*5,3])

    beta_a = np.array(beta_a)
    phi_a = np.array(phi_a)
    persev_a = np.array(persev_a)

    beta_b = np.array(beta_b)
    phi_b = np.array(phi_b)
    persev_b = np.array(persev_b)
    
    beta_a_sd = np.array(beta_a_sd)
    phi_a_sd = np.array(phi_a_sd)
    persev_a_sd = np.array(persev_a_sd)

    beta_b_sd = np.array(beta_b_sd)
    phi_b_sd = np.array(phi_b_sd)
    persev_b_sd = np.array(persev_b_sd)

    return beta_a,phi_a,persev_a,beta_b,phi_b,persev_b,beta_a_sd,phi_a_sd,persev_a_sd,beta_b_sd,phi_b_sd,persev_b_sd

def read_parameter_glm_tau(file_name): # read fitted parameters: beta_a,b;phi_a,b;persev_,b;gamma_a,b
    parameter_result = pd.read_csv(file_name)
    subject_num = 60
    beta_a = []
    phi_a = []
    persev_a = []

    beta_b = []
    phi_b = []
    persev_b = []
    tau = []

    for i in range(subject_num):
        beta_a.append(parameter_result.iloc[i,1])
    for i in range(subject_num):
        phi_a.append(parameter_result.iloc[i+subject_num,1])
    for i in range(subject_num):
        persev_a.append(parameter_result.iloc[i+subject_num*2,1])

    for i in range(subject_num):
        beta_b.append(parameter_result.iloc[i+subject_num*3,1])
    for i in range(subject_num):
        phi_b.append(parameter_result.iloc[i+subject_num*4,1])
    for i in range(subject_num):
        persev_b.append(parameter_result.iloc[i+subject_num*5,1])
    for i in range(subject_num):
        tau.append(parameter_result.iloc[i+subject_num*6,1])

    beta_a = np.array(beta_a)
    phi_a = np.array(phi_a)
    persev_a = np.array(persev_a)

    beta_b = np.array(beta_b)
    phi_b = np.array(phi_b)
    persev_b = np.array(persev_b)
    tau = np.array(tau)

    return beta_a,phi_a,persev_a,beta_b,phi_b,persev_b,tau

def read_parameter_glm(file_name): # read fitted parameters: beta_a,b;phi_a,b;persev_,b;gamma_a,b
    parameter_result = pd.read_csv(file_name)
    subject_num = 60
    beta_a = []
    phi_a = []
    persev_a = []
    gamma_a = []
    beta_b = []
    phi_b = []
    persev_b = []
    gamma_b = []
    for i in range(subject_num):
        beta_a.append(parameter_result.iloc[i,1])
    for i in range(subject_num):
        phi_a.append(parameter_result.iloc[i+subject_num,1])
    for i in range(subject_num):
        persev_a.append(parameter_result.iloc[i+subject_num*2,1])
    for i in range(subject_num):
        gamma_a.append(parameter_result.iloc[i+subject_num*3,1])
    for i in range(subject_num):
        beta_b.append(parameter_result.iloc[i+subject_num*4,1])
    for i in range(subject_num):
        phi_b.append(parameter_result.iloc[i+subject_num*5,1])
    for i in range(subject_num):
        persev_b.append(parameter_result.iloc[i+subject_num*6,1])
    for i in range(subject_num):
        gamma_b.append(parameter_result.iloc[i+subject_num*7,1])
    beta_a = np.array(beta_a)
    phi_a = np.array(phi_a)
    persev_a = np.array(persev_a)
    gamma_a = np.array(gamma_a)
    beta_b = np.array(beta_b)
    phi_b = np.array(phi_b)
    persev_b = np.array(persev_b)
    gamma_b = np.array(gamma_b)
    return beta_a,phi_a,persev_a,gamma_a,beta_b,phi_b,persev_b,gamma_b

def read_parameter_competition(file_name): # read fitted parameters: beta_a,b;phi_a,b;persev_,b;gamma_a,b
    parameter_result = pd.read_csv(file_name)
    subject_num = 60
    beta_a = []
    phi_a = []
    persev_a = []
    gamma_a = []

    for i in range(subject_num):
        beta_a.append(parameter_result.iloc[i,1])
    for i in range(subject_num):
        phi_a.append(parameter_result.iloc[i+subject_num,1])
    for i in range(subject_num):
        persev_a.append(parameter_result.iloc[i+subject_num*2,1])
    for i in range(subject_num):
        gamma_a.append(parameter_result.iloc[i+subject_num*3,1])

    beta_a = np.array(beta_a)
    phi_a = np.array(phi_a)
    persev_a = np.array(persev_a)
    gamma_a = np.array(gamma_a)

    return beta_a,phi_a,persev_a,gamma_a

def read_parameter_competition_sample(file_name): # read fitted parameters: beta_a,b;phi_a,b;persev_,b;gamma_a,b
    parameter_result = pd.read_csv(file_name)
    subject_num = 60
    beta_a = []
    phi_a = []
    persev_a = []
    gamma_a = []
    
    beta_a_sd = []
    phi_a_sd = []
    persev_a_sd = []
    gamma_a_sd = []

    for i in range(subject_num):
        beta_a.append(parameter_result.iloc[i,1])
    for i in range(subject_num):
        phi_a.append(parameter_result.iloc[i+subject_num,1])
    for i in range(subject_num):
        persev_a.append(parameter_result.iloc[i+subject_num*2,1])
    for i in range(subject_num):
        gamma_a.append(parameter_result.iloc[i+subject_num*3,1])
        
    for i in range(subject_num):
        beta_a_sd.append(parameter_result.iloc[i,3])
    for i in range(subject_num):
        phi_a_sd.append(parameter_result.iloc[i+subject_num,3])
    for i in range(subject_num):
        persev_a_sd.append(parameter_result.iloc[i+subject_num*2,3])
    for i in range(subject_num):
        gamma_a_sd.append(parameter_result.iloc[i+subject_num*3,3])

    beta_a = np.array(beta_a)
    phi_a = np.array(phi_a)
    persev_a = np.array(persev_a)
    gamma_a = np.array(gamma_a)
    
    beta_a_sd = np.array(beta_a_sd)
    phi_a_sd = np.array(phi_a_sd)
    persev_a_sd = np.array(persev_a_sd)
    gamma_a_sd = np.array(gamma_a_sd)

    return beta_a,phi_a,persev_a,gamma_a,beta_a_sd,phi_a_sd,persev_a_sd,gamma_a_sd


def read_parameter_glm2(file_name): # read fitted parameters: beta_a,b;phi_a,b;persev_,b;gamma_a,b
    parameter_result = pd.read_csv(file_name)
    subject_num = 60
    beta_a_0 = []
    phi_a_0 = []
    persev_a_0 = []
    gamma_a_0 = []
    
    beta_a_1 = []
    phi_a_1 = []
    persev_a_1 = []
    gamma_a_1 = []
    
    beta_b = []
    phi_b = []
    persev_b = []
    gamma_b = []
    for i in range(subject_num):
        beta_a_0.append(parameter_result.iloc[i,1])
    for i in range(subject_num):
        phi_a_0.append(parameter_result.iloc[i+subject_num,1])
    for i in range(subject_num):
        persev_a_0.append(parameter_result.iloc[i+subject_num*2,1])
    for i in range(subject_num):
        gamma_a_0.append(parameter_result.iloc[i+subject_num*3,1])
    for i in range(subject_num):
        beta_a_1.append(parameter_result.iloc[i+subject_num*4,1])
    for i in range(subject_num):
        phi_a_1.append(parameter_result.iloc[i+subject_num*5,1])
    for i in range(subject_num):
        persev_a_1.append(parameter_result.iloc[i+subject_num*6,1])
    for i in range(subject_num):
        gamma_a_1.append(parameter_result.iloc[i+subject_num*7,1])
    for i in range(subject_num):
        beta_b.append(parameter_result.iloc[i+subject_num*8,1])
    for i in range(subject_num):
        phi_b.append(parameter_result.iloc[i+subject_num*9,1])
    for i in range(subject_num):
        persev_b.append(parameter_result.iloc[i+subject_num*10,1])
    for i in range(subject_num):
        gamma_b.append(parameter_result.iloc[i+subject_num*11,1])
        
    beta_a_0 = np.array(beta_a_0)
    phi_a_0 = np.array(phi_a_0)
    persev_a_0 = np.array(persev_a_0)
    gamma_a_0 = np.array(gamma_a_0)
    
    beta_a_1 = np.array(beta_a_1)
    phi_a_1 = np.array(phi_a_1)
    persev_a_1 = np.array(persev_a_1)
    gamma_a_1 = np.array(gamma_a_1)
    
    beta_b = np.array(beta_b)
    phi_b = np.array(phi_b)
    persev_b = np.array(persev_b)
    gamma_b = np.array(gamma_b)
    return beta_a_0,phi_a_0,persev_a_0,gamma_a_0,beta_a_1,phi_a_1,persev_a_1,gamma_a_1,beta_b,phi_b,persev_b,gamma_b

def read_parameter_glm2_no_gamma(file_name): # read fitted parameters: beta_a,b;phi_a,b;persev_,b;gamma_a,b
    parameter_result = pd.read_csv(file_name)
    subject_num = 60
    beta_a_0 = []
    phi_a_0 = []
    persev_a_0 = []
    gamma_a_0 = []
    
    beta_a_1 = []
    phi_a_1 = []
    persev_a_1 = []
    gamma_a_1 = []
    
    beta_b = []

    for i in range(subject_num):
        beta_a_0.append(parameter_result.iloc[i,1])
    for i in range(subject_num):
        phi_a_0.append(parameter_result.iloc[i+subject_num,1])
    for i in range(subject_num):
        persev_a_0.append(parameter_result.iloc[i+subject_num*2,1])
    for i in range(subject_num):
        gamma_a_0.append(parameter_result.iloc[i+subject_num*3,1])
    for i in range(subject_num):
        beta_a_1.append(parameter_result.iloc[i+subject_num*4,1])
    for i in range(subject_num):
        phi_a_1.append(parameter_result.iloc[i+subject_num*5,1])
    for i in range(subject_num):
        persev_a_1.append(parameter_result.iloc[i+subject_num*6,1])
    for i in range(subject_num):
        gamma_a_1.append(parameter_result.iloc[i+subject_num*7,1])
    for i in range(subject_num):
        beta_b.append(parameter_result.iloc[i+subject_num*8,1])

        
    beta_a_0 = np.array(beta_a_0)
    phi_a_0 = np.array(phi_a_0)
    persev_a_0 = np.array(persev_a_0)
    gamma_a_0 = np.array(gamma_a_0)
    
    beta_a_1 = np.array(beta_a_1)
    phi_a_1 = np.array(phi_a_1)
    persev_a_1 = np.array(persev_a_1)
    gamma_a_1 = np.array(gamma_a_1)
    
    beta_b = np.array(beta_b)

    return beta_a_0,phi_a_0,persev_a_0,gamma_a_0,beta_a_1,phi_a_1,persev_a_1,gamma_a_1,beta_b



def read_parameter_glm2_no_gamma_para(file_name): # read fitted parameters: beta_a,b;phi_a,b;persev_,b;gamma_a,b
    parameter_result = pd.read_csv(file_name)
    subject_num = 60
    beta_a_0 = []
    phi_a_0 = []
    persev_a_0 = []
    gamma_a_0 = []
    
    beta_a_1 = []
    phi_a_1 = []


    for i in range(subject_num):
        beta_a_0.append(parameter_result.iloc[i,1])
    for i in range(subject_num):
        phi_a_0.append(parameter_result.iloc[i+subject_num,1])
    for i in range(subject_num):
        persev_a_0.append(parameter_result.iloc[i+subject_num*2,1])
    for i in range(subject_num):
        gamma_a_0.append(parameter_result.iloc[i+subject_num*3,1])
    for i in range(subject_num):
        beta_a_1.append(parameter_result.iloc[i+subject_num*4,1])
    for i in range(subject_num):
        phi_a_1.append(parameter_result.iloc[i+subject_num*5,1])


        
    beta_a_0 = np.array(beta_a_0)
    phi_a_0 = np.array(phi_a_0)
    persev_a_0 = np.array(persev_a_0)
    gamma_a_0 = np.array(gamma_a_0)
    
    beta_a_1 = np.array(beta_a_1)
    phi_a_1 = np.array(phi_a_1)


    return beta_a_0,phi_a_0,persev_a_0,gamma_a_0,beta_a_1,phi_a_1

def read_parameter_glm2_no_gamma_sample(file_name): # read fitted parameters: beta_a,b;phi_a,b;persev_,b;gamma_a,b
    parameter_result = pd.read_csv(file_name)
    subject_num = 60
    beta_a_0 = []
    phi_a_0 = []
    persev_a_0 = []
    gamma_a_0 = []
    beta_a_1 = []
    phi_a_1 = []
    persev_a_1 = []
    gamma_a_1 = []
    beta_b = []
    
    beta_a_0_sd = []
    phi_a_0_sd = []
    persev_a_0_sd = []
    gamma_a_0_sd = []
    beta_a_1_sd = []
    phi_a_1_sd = []
    persev_a_1_sd = []
    gamma_a_1_sd = []
    beta_b_sd = []

    for i in range(subject_num):
        beta_a_0.append(parameter_result.iloc[i,1])
    for i in range(subject_num):
        phi_a_0.append(parameter_result.iloc[i+subject_num,1])
    for i in range(subject_num):
        persev_a_0.append(parameter_result.iloc[i+subject_num*2,1])
    for i in range(subject_num):
        gamma_a_0.append(parameter_result.iloc[i+subject_num*3,1])
    for i in range(subject_num):
        beta_a_1.append(parameter_result.iloc[i+subject_num*4,1])
    for i in range(subject_num):
        phi_a_1.append(parameter_result.iloc[i+subject_num*5,1])
    for i in range(subject_num):
        persev_a_1.append(parameter_result.iloc[i+subject_num*6,1])
    for i in range(subject_num):
        gamma_a_1.append(parameter_result.iloc[i+subject_num*7,1])
    for i in range(subject_num):
        beta_b.append(parameter_result.iloc[i+subject_num*8,1])
        
    for i in range(subject_num):
        beta_a_0_sd.append(parameter_result.iloc[i,3])
    for i in range(subject_num):
        phi_a_0_sd.append(parameter_result.iloc[i+subject_num,3])
    for i in range(subject_num):
        persev_a_0_sd.append(parameter_result.iloc[i+subject_num*2,3])
    for i in range(subject_num):
        gamma_a_0_sd.append(parameter_result.iloc[i+subject_num*3,3])
    for i in range(subject_num):
        beta_a_1_sd.append(parameter_result.iloc[i+subject_num*4,3])
    for i in range(subject_num):
        phi_a_1_sd.append(parameter_result.iloc[i+subject_num*5,3])
    for i in range(subject_num):
        persev_a_1_sd.append(parameter_result.iloc[i+subject_num*6,3])
    for i in range(subject_num):
        gamma_a_1_sd.append(parameter_result.iloc[i+subject_num*7,3])
    for i in range(subject_num):
        beta_b_sd.append(parameter_result.iloc[i+subject_num*8,3])

        
    beta_a_0 = np.array(beta_a_0)
    phi_a_0 = np.array(phi_a_0)
    persev_a_0 = np.array(persev_a_0)
    gamma_a_0 = np.array(gamma_a_0)
    beta_a_1 = np.array(beta_a_1)
    phi_a_1 = np.array(phi_a_1)
    persev_a_1 = np.array(persev_a_1)
    gamma_a_1 = np.array(gamma_a_1)
    beta_b = np.array(beta_b)
    
    beta_a_0_sd = np.array(beta_a_0_sd)
    phi_a_0_sd = np.array(phi_a_0_sd)
    persev_a_0_sd = np.array(persev_a_0_sd)
    gamma_a_0_sd = np.array(gamma_a_0_sd)
    beta_a_1_sd = np.array(beta_a_1_sd)
    phi_a_1_sd = np.array(phi_a_1_sd)
    persev_a_1_sd = np.array(persev_a_1_sd)
    gamma_a_1_sd = np.array(gamma_a_1_sd)
    beta_b_sd = np.array(beta_b_sd)

    return beta_a_0,phi_a_0,persev_a_0,gamma_a_0,beta_a_1,phi_a_1,persev_a_1,gamma_a_1,beta_b,beta_a_0_sd,phi_a_0_sd,persev_a_0_sd,gamma_a_0_sd,beta_a_1_sd,phi_a_1_sd,persev_a_1_sd,gamma_a_1_sd,beta_b_sd


def read_parameter_glm2_no_gamma_beta(file_name): # read fitted parameters: beta_a,b;phi_a,b;persev_,b;gamma_a,b
    parameter_result = pd.read_csv(file_name)
    subject_num = 60
    beta_a_0 = []
    phi_a_0 = []
    persev_a_0 = []
    gamma_a_0 = []
    beta_a_1 = []


    for i in range(subject_num):
        beta_a_0.append(parameter_result.iloc[i,1])
    for i in range(subject_num):
        phi_a_0.append(parameter_result.iloc[i+subject_num,1])
    for i in range(subject_num):
        persev_a_0.append(parameter_result.iloc[i+subject_num*2,1])
    for i in range(subject_num):
        gamma_a_0.append(parameter_result.iloc[i+subject_num*3,1])
    for i in range(subject_num):
        beta_a_1.append(parameter_result.iloc[i+subject_num*4,1])

        
    beta_a_0 = np.array(beta_a_0)
    phi_a_0 = np.array(phi_a_0)
    persev_a_0 = np.array(persev_a_0)
    gamma_a_0 = np.array(gamma_a_0) 
    beta_a_1 = np.array(beta_a_1)

    return beta_a_0,phi_a_0,persev_a_0,gamma_a_0,beta_a_1


def read_parameter_glm2_no_gamma_beta_sample(file_name): # read fitted parameters: beta_a,b;phi_a,b;persev_,b;gamma_a,b
    parameter_result = pd.read_csv(file_name)
    subject_num = 60
    beta_a_0 = []
    phi_a_0 = []
    persev_a_0 = []
    gamma_a_0 = []
    beta_a_1 = []
    
    beta_a_0_sd = []
    phi_a_0_sd = []
    persev_a_0_sd = []
    gamma_a_0_sd = []
    beta_a_1_sd = []


    for i in range(subject_num):
        beta_a_0.append(parameter_result.iloc[i,1])
    for i in range(subject_num):
        phi_a_0.append(parameter_result.iloc[i+subject_num,1])
    for i in range(subject_num):
        persev_a_0.append(parameter_result.iloc[i+subject_num*2,1])
    for i in range(subject_num):
        gamma_a_0.append(parameter_result.iloc[i+subject_num*3,1])
    for i in range(subject_num):
        beta_a_1.append(parameter_result.iloc[i+subject_num*4,1])
        
    for i in range(subject_num):
        beta_a_0_sd.append(parameter_result.iloc[i,3])
    for i in range(subject_num):
        phi_a_0_sd.append(parameter_result.iloc[i+subject_num,3])
    for i in range(subject_num):
        persev_a_0_sd.append(parameter_result.iloc[i+subject_num*2,3])
    for i in range(subject_num):
        gamma_a_0_sd.append(parameter_result.iloc[i+subject_num*3,3])
    for i in range(subject_num):
        beta_a_1_sd.append(parameter_result.iloc[i+subject_num*4,3])

        
    beta_a_0 = np.array(beta_a_0)
    phi_a_0 = np.array(phi_a_0)
    persev_a_0 = np.array(persev_a_0)
    gamma_a_0 = np.array(gamma_a_0) 
    beta_a_1 = np.array(beta_a_1)
    
    beta_a_0_sd = np.array(beta_a_0_sd)
    phi_a_0_sd = np.array(phi_a_0_sd)
    persev_a_0_sd = np.array(persev_a_0_sd)
    gamma_a_0_sd = np.array(gamma_a_0_sd) 
    beta_a_1_sd = np.array(beta_a_1_sd)

    return beta_a_0,phi_a_0,persev_a_0,gamma_a_0,beta_a_1,beta_a_0_sd,phi_a_0_sd,persev_a_0_sd,gamma_a_0_sd,beta_a_1_sd


def read_parameter_glm_7_para(file_name): # read fitted parameters: beta_a,b;phi_a,b;persev_,b;gamma_a,b
    parameter_result = pd.read_csv(file_name)
    subject_num = 60
    beta_a = []
    phi_a = []
    persev_a = []
    gamma_a = []
    beta_b = []
    phi_b = []
    persev_b = []

    for i in range(subject_num):
        beta_a.append(parameter_result.iloc[i,1])
    for i in range(subject_num):
        phi_a.append(parameter_result.iloc[i+subject_num,1])
    for i in range(subject_num):
        persev_a.append(parameter_result.iloc[i+subject_num*2,1])
    for i in range(subject_num):
        gamma_a.append(parameter_result.iloc[i+subject_num*3,1])
    for i in range(subject_num):
        beta_b.append(parameter_result.iloc[i+subject_num*4,1])
    for i in range(subject_num):
        phi_b.append(parameter_result.iloc[i+subject_num*5,1])
    for i in range(subject_num):
        persev_b.append(parameter_result.iloc[i+subject_num*6,1])

    beta_a = np.array(beta_a)
    phi_a = np.array(phi_a)
    persev_a = np.array(persev_a)
    gamma_a = np.array(gamma_a)
    beta_b = np.array(beta_b)
    phi_b = np.array(phi_b)
    persev_b = np.array(persev_b)

    return beta_a,phi_a,persev_a,gamma_a,beta_b,phi_b,persev_b

def read_parameter_happiness(file_name): #read fitted parameters: beta,phi,persev,gamma,comparison_level
    parameter_result = pd.read_csv(file_name)
    subject_num = 60
    beta = []
    phi = []
    persev = []
    gamma = []
    comparison_level = []
    for i in range(subject_num):
        beta.append(parameter_result.iloc[i,1])
    for i in range(subject_num):
        phi.append(parameter_result.iloc[i+subject_num,1])
    for i in range(subject_num):
        persev.append(parameter_result.iloc[i+subject_num*2,1])
    for i in range(subject_num):
        gamma.append(parameter_result.iloc[i+subject_num*3,1])
    for i in range(subject_num):
        comparison_level.append(parameter_result.iloc[i+subject_num*4,1])
    beta = np.array(beta)
    phi = np.array(phi)
    persev = np.array(persev)
    gamma = np.array(gamma)
    comparison_level = np.array(comparison_level)
    return beta,phi,persev,gamma,comparison_level
def read_parameter_learning(file_name): #read fitted parameters: beta,phi,persev,gamma,comparison_level
    parameter_result = pd.read_csv(file_name)
    subject_num = 60
    beta = []
    phi = []
    persev = []
    gamma = []
    comparison_level = []
    for i in range(subject_num):
        beta.append(parameter_result.iloc[i,1])
    for i in range(subject_num):
        phi.append(parameter_result.iloc[i+subject_num,1])
    for i in range(subject_num):
        persev.append(parameter_result.iloc[i+subject_num*2,1])
    for i in range(subject_num):
        gamma.append(parameter_result.iloc[i+subject_num*3,1])
    for i in range(subject_num):
        comparison_level.append(parameter_result.iloc[i+subject_num*4,1])
    beta = np.array(beta)
    phi = np.array(phi)
    persev = np.array(persev)
    gamma = np.array(gamma)
    comparison_level = np.array(comparison_level)
    return beta,phi,persev,gamma,comparison_level

def read_parameter_nhb_delta(file_name): #read fitted parameters: beta,phi,persev,gamma,learning_rate
    parameter_result = pd.read_csv(file_name)
    subject_num = 60
    beta = []
    phi = []
    persev = []
    gamma = []
    learning_rate = []
    for i in range(subject_num):
        beta.append(parameter_result.iloc[i,1])
    for i in range(subject_num):
        phi.append(parameter_result.iloc[i+subject_num,1])
    for i in range(subject_num):
        persev.append(parameter_result.iloc[i+subject_num*2,1])
    for i in range(subject_num):
        gamma.append(parameter_result.iloc[i+subject_num*3,1])
    for i in range(subject_num):
        learning_rate.append(parameter_result.iloc[i+subject_num*4,1])
    beta = np.array(beta)
    phi = np.array(phi)
    persev = np.array(persev)
    gamma = np.array(gamma)
    learning_rate = np.array(learning_rate)
    return beta,phi,persev,gamma,learning_rate

def read_parameter_no_gamma(file_name): # read fitted parameters: beta,phi,persev,gamma
    parameter_result = pd.read_csv(file_name)
    subject_num = 60
    beta = []
    phi = []
    persev = []
    for i in range(subject_num):
        beta.append(parameter_result.iloc[i,1])
    for i in range(subject_num):
        phi.append(parameter_result.iloc[i+subject_num,1])
    for i in range(subject_num):
        persev.append(parameter_result.iloc[i+subject_num*2,1])
    beta = np.array(beta)
    phi = np.array(phi)
    persev = np.array(persev)
    return beta,phi,persev

def read_parameter_no_gamma_sample(file_name): # read fitted parameters: beta,phi,persev,gamma
    parameter_result = pd.read_csv(file_name)
    subject_num = 60
    beta = []
    phi = []
    persev = []
    
    beta_sd = []
    phi_sd = []
    persev_sd = []
    
    for i in range(subject_num):
        beta.append(parameter_result.iloc[i,1])
    for i in range(subject_num):
        phi.append(parameter_result.iloc[i+subject_num,1])
    for i in range(subject_num):
        persev.append(parameter_result.iloc[i+subject_num*2,1])
        
    for i in range(subject_num):
        beta_sd.append(parameter_result.iloc[i,3])
    for i in range(subject_num):
        phi_sd.append(parameter_result.iloc[i+subject_num,3])
    for i in range(subject_num):
        persev_sd.append(parameter_result.iloc[i+subject_num*2,3])
        
    beta_sd = np.array(beta_sd)
    phi_sd = np.array(phi_sd)
    persev_sd = np.array(persev_sd)
    
    beta = np.array(beta)
    phi = np.array(phi)
    persev = np.array(persev)
    
    return beta,phi,persev,beta_sd,phi_sd,persev_sd

def read_parameter_no_gamma_no_persev(file_name): # read fitted parameters: beta,phi,persev,gamma
    parameter_result = pd.read_csv(file_name)
    subject_num = 60
    beta = []
    phi = []
    for i in range(subject_num):
        beta.append(parameter_result.iloc[i,1])
    for i in range(subject_num):
        phi.append(parameter_result.iloc[i+subject_num,1])
    beta = np.array(beta)
    phi = np.array(phi)
    return beta,phi

def read_parameter_no_gamma_no_persev_sample(file_name): # read fitted parameters: beta,phi,persev,gamma
    parameter_result = pd.read_csv(file_name)
    subject_num = 60
    beta = []
    phi = []
    
    beta_sd = []
    phi_sd = []
    
    for i in range(subject_num):
        beta.append(parameter_result.iloc[i,1])
    for i in range(subject_num):
        phi.append(parameter_result.iloc[i+subject_num,1])
        
    for i in range(subject_num):
        beta_sd.append(parameter_result.iloc[i,3])
    for i in range(subject_num):
        phi_sd.append(parameter_result.iloc[i+subject_num,3])
        
    beta = np.array(beta)
    phi = np.array(phi)
    
    beta_sd = np.array(beta_sd)
    phi_sd = np.array(phi_sd)
    
    return beta,phi,beta_sd,phi_sd

def read_parameter_no_gamma_no_persev_no_phi(file_name): # read fitted parameters: beta,phi,persev,gamma
    parameter_result = pd.read_csv(file_name)
    subject_num = 60
    beta = []
    for i in range(subject_num):
        beta.append(parameter_result.iloc[i,1])
    beta = np.array(beta)
    return beta

def read_parameter_no_gamma_no_persev_no_phi_sample(file_name): # read fitted parameters: beta,phi,persev,gamma
    parameter_result = pd.read_csv(file_name)
    subject_num = 60
    beta = []
    beta_sd = []
    for i in range(subject_num):
        beta.append(parameter_result.iloc[i,1])
    for i in range(subject_num):
        beta_sd.append(parameter_result.iloc[i,3])
    beta = np.array(beta)
    beta_sd = np.array(beta_sd)
    return beta,beta_sd

def read_parameter_glm_no_gamma_para(file_name): # read fitted parameters: beta,phi,persev,gamma
    parameter_result = pd.read_csv(file_name)
    subject_num = 60
    beta = []
    phi = []
    persev = []
    beta_a = []
    for i in range(subject_num):
        beta.append(parameter_result.iloc[i,1])
    for i in range(subject_num):
        phi.append(parameter_result.iloc[i+subject_num,1])
    for i in range(subject_num):
        persev.append(parameter_result.iloc[i+subject_num*2,1])
    for i in range(subject_num):
        beta_a.append(parameter_result.iloc[i+subject_num*3,1])
    beta = np.array(beta)
    phi = np.array(phi)
    persev = np.array(persev)
    beta_a = np.array(beta_a)
    return beta,phi,persev,beta_a

def read_parameter_glm_no_gamma_para_sample(file_name): # read fitted parameters: beta,phi,persev,gamma
    parameter_result = pd.read_csv(file_name)
    subject_num = 60
    beta = []
    phi = []
    persev = []
    beta_a = []
    
    beta_sd = []
    phi_sd = []
    persev_sd = []
    beta_a_sd = []
    
    for i in range(subject_num):
        beta.append(parameter_result.iloc[i,1])
    for i in range(subject_num):
        phi.append(parameter_result.iloc[i+subject_num,1])
    for i in range(subject_num):
        persev.append(parameter_result.iloc[i+subject_num*2,1])
    for i in range(subject_num):
        beta_a.append(parameter_result.iloc[i+subject_num*3,1])
        
    for i in range(subject_num):
        beta_sd.append(parameter_result.iloc[i,3])
    for i in range(subject_num):
        phi_sd.append(parameter_result.iloc[i+subject_num,3])
    for i in range(subject_num):
        persev_sd.append(parameter_result.iloc[i+subject_num*2,3])
    for i in range(subject_num):
        beta_a_sd.append(parameter_result.iloc[i+subject_num*3,3])
    
    beta = np.array(beta)
    phi = np.array(phi)
    persev = np.array(persev)
    beta_a = np.array(beta_a)
    
    beta_sd = np.array(beta_sd)
    phi_sd = np.array(phi_sd)
    persev_sd = np.array(persev_sd)
    beta_a_sd = np.array(beta_a_sd)
    
    return beta,phi,persev,beta_a,beta_sd,phi_sd,persev_sd,beta_a_sd

In [19]:
# calculate the log likelyhood
def action_probability(Q,action):
    return np.log(np.exp(Q[action])/(np.exp(Q[0])+np.exp(Q[1])+np.exp(Q[2])+np.exp(Q[3])))
def rewards2comparison(action,reward,reward_B,gamma):
    comparison = []
    com = 0
    for i in range(len(action)):
        if action[i]!=4:
            comparison.append(com)
            com = reward[i] - reward_B[i] + gamma*com
        else:
            comparison.append(0)
            com = gamma*com
    return comparison
def rewards2comparison_sqrt(action,reward,reward_B,gamma):
    comparison = []
    com = 0
    for i in range(len(action)):
        if action[i]!=4:
            if com<0:
                comparison.append(-np.sqrt(-com))
            else:
                comparison.append(np.sqrt(com))
            com = reward[i] - reward_B[i] + gamma*com
        else:
            comparison.append(0)
            com = gamma*com
    return comparison

def rewards2comparison_sqrt_competition_learn(action,reward,reward_B,gamma):
    comparison = []
    com = 0
    for i in range(len(action)):
        if action[i]!=4:
            com = reward[i] - reward_B[i] + gamma*com
            if com<0:
                comparison.append(-np.sqrt(-com))
            else:
                comparison.append(np.sqrt(com))
        else:
            comparison.append(0)
            com = gamma*com
    return comparison

def log_likelyhood_elife(file_name):#elife
    beta,phi,persev,gamma = read_parameter(file_name)
    subjects = 60
    trials = 150
    subject_loglikelyhood = []
    
    for s in range(subjects):
        action,reward,reward_B,_ =read_data(s) 

        comparision = rewards2comparison(action,reward,reward_B,0)
        loglikelyhood = 0
        sig_o = 4
        sig_d = 2.8
        v = np.ones((4))*50
        sig = np.ones((4))*4
        for t in range(trials):
            if action[t]!=4:
                #beta:temperature; v:action value; eb:exploration bonus; pb: perseverance bonus; re: random exploration
                eb = phi[s] * sig
                pb = np.zeros((4))
                re = gamma[s] * v / np.sum(sig)
                if t>0:
                    if action[t-1]!=4:
                        pb[action[t-1]] = persev[s]
                # elife: beta[s]*(v+eb+pb+re); nhb: (beta[s]*v+eb+pb+re)
                loglikelyhood += action_probability(beta[s]*(v+eb+pb+re),action[t])
                pe = reward[t] - v[action[t]]
                Kgain = np.square(sig[action[t]])/(np.square(sig[action[t]])+np.square(sig_o))
                v[action[t]] = v[action[t]] + Kgain * pe
                sig[action[t]] = np.sqrt((1-Kgain)*np.square(sig[action[t]]))
            sig = np.sqrt(np.square(sig)+np.square(sig_d))
        subject_loglikelyhood.append(loglikelyhood)
    return subject_loglikelyhood
def log_likelyhood_nhb(file_name):#nhb
    beta,phi,persev,gamma = read_parameter(file_name)
    subjects = 60
    trials = 150
    subject_loglikelyhood = []
    for s in range(subjects):
        action,reward,reward_B,_ =read_data(s) 
        
        comparision = rewards2comparison(action,reward,reward_B,0)
        loglikelyhood = 0
        sig_o = 4
        sig_d = 2.8
        v = np.ones((4))*50
        sig = np.ones((4))*4
        for t in range(trials):
            if action[t]!=4:
                #beta:temperature; v:action value; eb:exploration bonus; pb: perseverance bonus; re: random exploration
                eb = phi[s] * sig
                pb = np.zeros((4))
                re = gamma[s] * v / np.sum(sig)
                if t>0:
                    if action[t-1]!=4:
                        pb[action[t-1]] = persev[s]
                # elife: beta[s]*(v+eb+pb+re); nhb: (beta[s]*v+eb+pb+re)
                loglikelyhood += action_probability((beta[s]*v+eb+pb+re),action[t])
                pe = reward[t] - v[action[t]]
                Kgain = np.square(sig[action[t]])/(np.square(sig[action[t]])+np.square(sig_o))
                v[action[t]] = v[action[t]] + Kgain * pe
                sig[action[t]] = np.sqrt((1-Kgain)*np.square(sig[action[t]]))
            sig = np.sqrt(np.square(sig)+np.square(sig_d))
        subject_loglikelyhood.append(loglikelyhood)
    return subject_loglikelyhood

def log_likelyhood_nhb_c(file_name):#nhb
    beta,phi,persev,gamma = read_parameter(file_name)
    subjects = 60
    trials = 150
    subject_loglikelyhood = []
    for s in range(subjects):
        action,reward,reward_B,_ =read_data(s) 
        
        comparision = rewards2comparison(action,reward,reward_B,0)
        loglikelyhood = 0
        sig_o = 4
        sig_d = 2.8
        v = np.ones((4))*50
        sig = np.ones((4))*4
        for t in range(trials):
            if action[t]!=4:
                #beta:temperature; v:action value; eb:exploration bonus; pb: perseverance bonus; re: random exploration
                eb = phi[s] * sig
                pb = np.zeros((4))
                re = gamma[s] * np.ones((4))
                if t>0:
                    if action[t-1]!=4:
                        pb[action[t-1]] = persev[s]
                # elife: beta[s]*(v+eb+pb+re); nhb: (beta[s]*v+eb+pb+re)
                loglikelyhood += action_probability((beta[s]*v+eb+pb+re),action[t])
                pe = reward[t] - v[action[t]]
                Kgain = np.square(sig[action[t]])/(np.square(sig[action[t]])+np.square(sig_o))
                v[action[t]] = v[action[t]] + Kgain * pe
                sig[action[t]] = np.sqrt((1-Kgain)*np.square(sig[action[t]]))
            sig = np.sqrt(np.square(sig)+np.square(sig_d))
        subject_loglikelyhood.append(loglikelyhood)
    return subject_loglikelyhood

In [21]:
def loo_glm_vep_test(file_name):#nhb
    beta_a_mean,phi_a_mean,persev_a_mean,beta_b_mean,phi_b_mean,persev_b_mean,beta_a_sd,phi_a_sd,persev_a_sd,beta_b_sd,phi_b_sd,persev_b_sd = read_parameter_glm_3_para_sample(file_name)
    subjects = 60
    trials = 150
    subject_loo = []
    sample_num = 100
    
    
    for i in range(subjects):
        beta_a = np.random.normal(beta_a_mean[i], 0, size=sample_num)
        phi_a = np.random.normal(phi_a_mean[i], 0, size=sample_num)
        persev_a = np.random.normal(persev_a_mean[i], 0, size=sample_num)
        beta_b = np.random.normal(beta_b_mean[i], 0, size=sample_num)
        phi_b = np.random.normal(phi_b_mean[i], 0, size=sample_num)
        persev_b = np.random.normal(persev_b_mean[i], 0, size=sample_num)
        
        action,reward,reward_B,_ =read_data(i) 
        comparision = rewards2comparison_sqrt(action,reward,reward_B,0)
        
        subject_loglikelyhood = []

        for s in range(sample_num):
            sample_loglikelyhood = []
            # loglikelyhood = 0
            sig_o = 4
            sig_d = 2.8
            v = np.ones((4))*50
            sig = np.ones((4))*4
            for t in range(trials):
                if action[t]!=4:
                    #beta:temperature; v:action value; eb:exploration bonus; pb: perseverance bonus; re: random exploration
                    min_sig = np.min(sig)*np.ones((4))
                    re_sig = sig-min_sig
                    eb = (phi_a[s]*comparision[t]+phi_b[s]) * re_sig
                    pb = np.zeros((4))
                    if t>0:
                        if action[t-1]!=4:
                            pb[action[t-1]] = (persev_a[s]*comparision[t]+persev_b[s])
                    # elife: beta[s]*(v+eb+pb+re); nhb: (beta[s]*v+eb+pb+re)
                    beta_t = beta_a[s]*comparision[t]+beta_b[s]
                    if beta_t<0:
                        beta_t = 0
                    sample_loglikelyhood.append(action_probability((beta_t*v+eb+pb),action[t]))
                    pe = reward[t] - v[action[t]]
                    Kgain = np.square(sig[action[t]])/(np.square(sig[action[t]])+np.square(sig_o))
                    v[action[t]] = v[action[t]] + Kgain * pe
                    sig[action[t]] = np.sqrt((1-Kgain)*np.square(sig[action[t]]))
                else:
                    sample_loglikelyhood.append(0)
                sig = np.sqrt(np.square(sig)+np.square(sig_d))
            subject_loglikelyhood.append(sample_loglikelyhood)
        subject_loglikelyhood = np.array(subject_loglikelyhood)
        loo,_,_ = psisloo(subject_loglikelyhood)
        subject_loo.append(loo)
    return subject_loo

def loo_glm_vep(file_name):#nhb
    beta_a_mean,phi_a_mean,persev_a_mean,beta_b_mean,phi_b_mean,persev_b_mean,beta_a_sd,phi_a_sd,persev_a_sd,beta_b_sd,phi_b_sd,persev_b_sd = read_parameter_glm_3_para_sample(file_name)
    subjects = 60
    trials = 150
    subject_loo = []
    sample_num = 100
    
    
    for i in range(subjects):
        beta_a = np.random.normal(beta_a_mean[i], beta_a_sd[i], size=sample_num)
        phi_a = np.random.normal(phi_a_mean[i], phi_a_sd[i], size=sample_num)
        persev_a = np.random.normal(persev_a_mean[i], persev_a_sd[i], size=sample_num)
        beta_b = np.random.normal(beta_b_mean[i], beta_b_sd[i], size=sample_num)
        phi_b = np.random.normal(phi_b_mean[i], phi_b_sd[i], size=sample_num)
        persev_b = np.random.normal(persev_b_mean[i], persev_b_sd[i], size=sample_num)
        
        action,reward,reward_B,_ =read_data(i) 
        comparision = rewards2comparison_sqrt(action,reward,reward_B,0)
        
        subject_loglikelyhood = []

        for s in range(sample_num):
            sample_loglikelyhood = []
            # loglikelyhood = 0
            sig_o = 4
            sig_d = 2.8
            v = np.ones((4))*50
            sig = np.ones((4))*4
            for t in range(trials):
                if action[t]!=4:
                    #beta:temperature; v:action value; eb:exploration bonus; pb: perseverance bonus; re: random exploration
                    min_sig = np.min(sig)*np.ones((4))
                    re_sig = sig-min_sig
                    eb = (phi_a[s]*comparision[t]+phi_b[s]) * re_sig
                    pb = np.zeros((4))
                    if t>0:
                        if action[t-1]!=4:
                            pb[action[t-1]] = (persev_a[s]*comparision[t]+persev_b[s])
                    # elife: beta[s]*(v+eb+pb+re); nhb: (beta[s]*v+eb+pb+re)
                    beta_t = beta_a[s]*comparision[t]+beta_b[s]
                    if beta_t<0:
                        beta_t = 0
                    sample_loglikelyhood.append(action_probability((beta_t*v+eb+pb),action[t]))
                    pe = reward[t] - v[action[t]]
                    Kgain = np.square(sig[action[t]])/(np.square(sig[action[t]])+np.square(sig_o))
                    v[action[t]] = v[action[t]] + Kgain * pe
                    sig[action[t]] = np.sqrt((1-Kgain)*np.square(sig[action[t]]))
                else:
                    sample_loglikelyhood.append(0)
                sig = np.sqrt(np.square(sig)+np.square(sig_d))
            subject_loglikelyhood.append(sample_loglikelyhood)
        subject_loglikelyhood = np.array(subject_loglikelyhood)
        loo,_,_ = psisloo(subject_loglikelyhood)
        subject_loo.append(loo)
    return subject_loo

def loo_glm_v(file_name):#nhb
    beta_a_mean,beta_b_mean,phi_b_mean,persev_b_mean,beta_a_sd,beta_b_sd,phi_b_sd,persev_b_sd = read_parameter_glm_no_gamma_para_sample(file_name)
    subjects = 60
    trials = 150
    subject_loo = []
    sample_num = 100
    
    for i in range(subjects):
        beta_a = np.random.normal(beta_a_mean[i], beta_a_sd[i], size=sample_num)

        beta_b = np.random.normal(beta_b_mean[i], beta_b_sd[i], size=sample_num)
        phi_b = np.random.normal(phi_b_mean[i], phi_b_sd[i], size=sample_num)
        persev_b = np.random.normal(persev_b_mean[i], persev_b_sd[i], size=sample_num)
        
        action,reward,reward_B,_ =read_data(i) 
        comparision = rewards2comparison_sqrt(action,reward,reward_B,0)
        
        subject_loglikelyhood = []
    
        for s in range(sample_num):

            sample_loglikelyhood = []
            sig_o = 4
            sig_d = 2.8
            v = np.ones((4))*50
            sig = np.ones((4))*4
            for t in range(trials):
                if action[t]!=4:
                    #beta:temperature; v:action value; eb:exploration bonus; pb: perseverance bonus; re: random exploration
                    min_sig = np.min(sig)*np.ones((4))
                    re_sig = sig-min_sig
                    eb = phi_b[s] * re_sig
                    pb = np.zeros((4))
                    if t>0:
                        if action[t-1]!=4:
                            pb[action[t-1]] = persev_b[s]
                    # elife: beta[s]*(v+eb+pb+re); nhb: (beta[s]*v+eb+pb+re)
                    beta_t = beta_a[s]*comparision[t]+beta_b[s]
                    if beta_t<0:
                        beta_t = 0
                    sample_loglikelyhood.append(action_probability((beta_t*v+eb+pb),action[t]))
                    pe = reward[t] - v[action[t]]
                    Kgain = np.square(sig[action[t]])/(np.square(sig[action[t]])+np.square(sig_o))
                    v[action[t]] = v[action[t]] + Kgain * pe
                    sig[action[t]] = np.sqrt((1-Kgain)*np.square(sig[action[t]]))
                else:
                    sample_loglikelyhood.append(0)
                sig = np.sqrt(np.square(sig)+np.square(sig_d))
            subject_loglikelyhood.append(sample_loglikelyhood)
        subject_loglikelyhood = np.array(subject_loglikelyhood)
        loo,_,_ = psisloo(subject_loglikelyhood)
        subject_loo.append(loo)
    return subject_loo

def loo_glm_e(file_name):#nhb
    phi_a_mean,beta_b_mean,phi_b_mean,persev_b_mean,phi_a_sd,beta_b_sd,phi_b_sd,persev_b_sd = read_parameter_glm_no_gamma_para_sample(file_name)
    subjects = 60
    trials = 150
    subject_loo = []
    sample_num = 100
    
    for i in range(subjects):
        phi_a = np.random.normal(phi_a_mean[i], phi_a_sd[i], size=sample_num)

        beta_b = np.random.normal(beta_b_mean[i], beta_b_sd[i], size=sample_num)
        phi_b = np.random.normal(phi_b_mean[i], phi_b_sd[i], size=sample_num)
        persev_b = np.random.normal(persev_b_mean[i], persev_b_sd[i], size=sample_num)
        
        action,reward,reward_B,_ =read_data(i) 
        comparision = rewards2comparison_sqrt(action,reward,reward_B,0)
        
        subject_loglikelyhood = []
    
        for s in range(sample_num):

            sample_loglikelyhood = []
            sig_o = 4
            sig_d = 2.8
            v = np.ones((4))*50
            sig = np.ones((4))*4
            for t in range(trials):
                if action[t]!=4:
                    #beta:temperature; v:action value; eb:exploration bonus; pb: perseverance bonus; re: random exploration
                    min_sig = np.min(sig)*np.ones((4))
                    re_sig = sig-min_sig
                    eb = (phi_a[s]*comparision[t]+phi_b[s]) * re_sig
                    pb = np.zeros((4))
                    if t>0:
                        if action[t-1]!=4:
                            pb[action[t-1]] = persev_b[s]
                    # elife: beta[s]*(v+eb+pb+re); nhb: (beta[s]*v+eb+pb+re)
                    sample_loglikelyhood.append(action_probability((beta_b[s]*v+eb+pb),action[t]))
                    pe = reward[t] - v[action[t]]
                    Kgain = np.square(sig[action[t]])/(np.square(sig[action[t]])+np.square(sig_o))
                    v[action[t]] = v[action[t]] + Kgain * pe
                    sig[action[t]] = np.sqrt((1-Kgain)*np.square(sig[action[t]]))
                else:
                    sample_loglikelyhood.append(0)
                sig = np.sqrt(np.square(sig)+np.square(sig_d))
            subject_loglikelyhood.append(sample_loglikelyhood)
        subject_loglikelyhood = np.array(subject_loglikelyhood)
        loo,_,_ = psisloo(subject_loglikelyhood)
        subject_loo.append(loo)
    return subject_loo

def loo_glm_p(file_name):#nhb
    persev_a_mean,beta_b_mean,phi_b_mean,persev_b_mean,persev_a_sd,beta_b_sd,phi_b_sd,persev_b_sd = read_parameter_glm_no_gamma_para_sample(file_name)
    subjects = 60
    trials = 150
    subject_loo = []
    sample_num = 100
    
    for i in range(subjects):
        persev_a = np.random.normal(persev_a_mean[i], persev_a_sd[i], size=sample_num)

        beta_b = np.random.normal(beta_b_mean[i], beta_b_sd[i], size=sample_num)
        phi_b = np.random.normal(phi_b_mean[i], phi_b_sd[i], size=sample_num)
        persev_b = np.random.normal(persev_b_mean[i], persev_b_sd[i], size=sample_num)
        
        action,reward,reward_B,_ =read_data(i) 
        comparision = rewards2comparison_sqrt(action,reward,reward_B,0)
        
        subject_loglikelyhood = []
    
        for s in range(sample_num):

            sample_loglikelyhood = []
            sig_o = 4
            sig_d = 2.8
            v = np.ones((4))*50
            sig = np.ones((4))*4
            for t in range(trials):
                if action[t]!=4:
                    #beta:temperature; v:action value; eb:exploration bonus; pb: perseverance bonus; re: random exploration
                    min_sig = np.min(sig)*np.ones((4))
                    re_sig = sig-min_sig
                    eb = phi_b[s] * re_sig
                    pb = np.zeros((4))
                    if t>0:
                        if action[t-1]!=4:
                            pb[action[t-1]] = persev_a[s]*comparision[t]+persev_b[s]
                    # elife: beta[s]*(v+eb+pb+re); nhb: (beta[s]*v+eb+pb+re)
                    sample_loglikelyhood.append(action_probability((beta_b[s]*v+eb+pb),action[t]))
                    pe = reward[t] - v[action[t]]
                    Kgain = np.square(sig[action[t]])/(np.square(sig[action[t]])+np.square(sig_o))
                    v[action[t]] = v[action[t]] + Kgain * pe
                    sig[action[t]] = np.sqrt((1-Kgain)*np.square(sig[action[t]]))
                else:
                    sample_loglikelyhood.append(0)
                sig = np.sqrt(np.square(sig)+np.square(sig_d))
            subject_loglikelyhood.append(sample_loglikelyhood)
        subject_loglikelyhood = np.array(subject_loglikelyhood)
        loo,_,_ = psisloo(subject_loglikelyhood)
        subject_loo.append(loo)
    return subject_loo

def loo_vep(file_name):#nhb
    beta_b_mean,phi_b_mean,persev_b_mean,beta_b_sd,phi_b_sd,persev_b_sd = read_parameter_no_gamma_sample(file_name)
    subjects = 60
    trials = 150
    subject_loo = []
    sample_num = 100
    
    for i in range(subjects):
        beta_b = np.random.normal(beta_b_mean[i], beta_b_sd[i], size=sample_num)
        phi_b = np.random.normal(phi_b_mean[i], phi_b_sd[i], size=sample_num)
        persev_b = np.random.normal(persev_b_mean[i], persev_b_sd[i], size=sample_num)
        
        action,reward,reward_B,_ =read_data(i) 
        comparision = rewards2comparison_sqrt(action,reward,reward_B,0)
        
        subject_loglikelyhood = []
    
        for s in range(sample_num):

            sample_loglikelyhood = []
            sig_o = 4
            sig_d = 2.8
            v = np.ones((4))*50
            sig = np.ones((4))*4
            for t in range(trials):
                if action[t]!=4:
                    #beta:temperature; v:action value; eb:exploration bonus; pb: perseverance bonus; re: random exploration
                    min_sig = np.min(sig)*np.ones((4))
                    re_sig = sig-min_sig
                    eb = phi_b[s] * re_sig
                    pb = np.zeros((4))
                    if t>0:
                        if action[t-1]!=4:
                            pb[action[t-1]] = persev_b[s]
                    # elife: beta[s]*(v+eb+pb+re); nhb: (beta[s]*v+eb+pb+re)
                    sample_loglikelyhood.append(action_probability((beta_b[s]*v+eb+pb),action[t]))
                    pe = reward[t] - v[action[t]]
                    Kgain = np.square(sig[action[t]])/(np.square(sig[action[t]])+np.square(sig_o))
                    v[action[t]] = v[action[t]] + Kgain * pe
                    sig[action[t]] = np.sqrt((1-Kgain)*np.square(sig[action[t]]))
                else:
                    sample_loglikelyhood.append(0)
                sig = np.sqrt(np.square(sig)+np.square(sig_d))
            subject_loglikelyhood.append(sample_loglikelyhood)
        subject_loglikelyhood = np.array(subject_loglikelyhood)
        loo,_,_ = psisloo(subject_loglikelyhood)
        subject_loo.append(loo)
    return subject_loo

def loo_ve(file_name):#nhb
    beta_b_mean,phi_b_mean,beta_b_sd,phi_b_sd = read_parameter_no_gamma_no_persev_sample(file_name)
    subjects = 60
    trials = 150
    subject_loo = []
    sample_num = 100
    
    for i in range(subjects):
        beta_b = np.random.normal(beta_b_mean[i], beta_b_sd[i], size=sample_num)
        phi_b = np.random.normal(phi_b_mean[i], phi_b_sd[i], size=sample_num)
        
        action,reward,reward_B,_ =read_data(i) 
        comparision = rewards2comparison_sqrt(action,reward,reward_B,0)
        
        subject_loglikelyhood = []
    
        for s in range(sample_num):

            sample_loglikelyhood = []
            sig_o = 4
            sig_d = 2.8
            v = np.ones((4))*50
            sig = np.ones((4))*4
            for t in range(trials):
                if action[t]!=4:
                    #beta:temperature; v:action value; eb:exploration bonus; pb: perseverance bonus; re: random exploration
                    min_sig = np.min(sig)*np.ones((4))
                    re_sig = sig-min_sig
                    eb = phi_b[s] * re_sig

                    # elife: beta[s]*(v+eb+pb+re); nhb: (beta[s]*v+eb+pb+re)
                    sample_loglikelyhood.append(action_probability((beta_b[s]*v+eb),action[t]))
                    pe = reward[t] - v[action[t]]
                    Kgain = np.square(sig[action[t]])/(np.square(sig[action[t]])+np.square(sig_o))
                    v[action[t]] = v[action[t]] + Kgain * pe
                    sig[action[t]] = np.sqrt((1-Kgain)*np.square(sig[action[t]]))
                else:
                    sample_loglikelyhood.append(0)
                sig = np.sqrt(np.square(sig)+np.square(sig_d))
            subject_loglikelyhood.append(sample_loglikelyhood)
        subject_loglikelyhood = np.array(subject_loglikelyhood)
        loo,_,_ = psisloo(subject_loglikelyhood)
        subject_loo.append(loo)
    return subject_loo

def loo_v(file_name):#nhb
    beta_b_mean,beta_b_sd = read_parameter_no_gamma_no_persev_no_phi_sample(file_name)
    subjects = 60
    trials = 150
    subject_loo = []
    sample_num = 100
    
    for i in range(subjects):
        beta_b = np.random.normal(beta_b_mean[i], beta_b_sd[i], size=sample_num)
        
        action,reward,reward_B,_ =read_data(i) 
        comparision = rewards2comparison_sqrt(action,reward,reward_B,0)
        
        subject_loglikelyhood = []
    
        for s in range(sample_num):

            sample_loglikelyhood = []
            sig_o = 4
            sig_d = 2.8
            v = np.ones((4))*50
            sig = np.ones((4))*4
            for t in range(trials):
                if action[t]!=4:
                    #beta:temperature; v:action value; eb:exploration bonus; pb: perseverance bonus; re: random exploration

                    # elife: beta[s]*(v+eb+pb+re); nhb: (beta[s]*v+eb+pb+re)
                    sample_loglikelyhood.append(action_probability((beta_b[s]*v),action[t]))
                    pe = reward[t] - v[action[t]]
                    Kgain = np.square(sig[action[t]])/(np.square(sig[action[t]])+np.square(sig_o))
                    v[action[t]] = v[action[t]] + Kgain * pe
                    sig[action[t]] = np.sqrt((1-Kgain)*np.square(sig[action[t]]))
                else:
                    sample_loglikelyhood.append(0)
                sig = np.sqrt(np.square(sig)+np.square(sig_d))
            subject_loglikelyhood.append(sample_loglikelyhood)
        subject_loglikelyhood = np.array(subject_loglikelyhood)
        loo,_,_ = psisloo(subject_loglikelyhood)
        subject_loo.append(loo)
    return subject_loo

def loo_sup_delta(file_name):
    beta_b_mean,phi_b_mean,persev_b_mean,alpha_mean,beta_b_sd,phi_b_sd,persev_b_sd,alpha_sd = read_parameter_sup_sample(file_name)
    subjects = 60
    trials = 150
    subject_loo = []
    sample_num = 100
    
    for i in range(subjects):
        beta_b = np.random.normal(beta_b_mean[i], beta_b_sd[i], size=sample_num)
        phi_b = np.random.normal(phi_b_mean[i], phi_b_sd[i], size=sample_num)
        persev_b = np.random.normal(persev_b_mean[i], persev_b_sd[i], size=sample_num)
        alpha = np.random.normal(alpha_mean[i], alpha_sd[i], size=sample_num)
        
        action,reward,reward_B,_ =read_data(i) 
        comparision = rewards2comparison_sqrt(action,reward,reward_B,0)
        
        subject_loglikelyhood = []
    
        for s in range(sample_num):

            sample_loglikelyhood = []
            sig_o = 4
            sig_d = 2.8
            v = np.ones((4))*50
            sig = np.ones((4))*4
            for t in range(trials):
                if action[t]!=4:
                    #beta:temperature; v:action value; eb:exploration bonus; pb: perseverance bonus; re: random exploration
                    min_sig = np.min(sig)*np.ones((4))
                    re_sig = sig-min_sig
                    eb = phi_b[s] * re_sig
                    pb = np.zeros((4))
                    if t>0:
                        if action[t-1]!=4:
                            pb[action[t-1]] = persev_b[s]
                    # elife: beta[s]*(v+eb+pb+re); nhb: (beta[s]*v+eb+pb+re)
                    sample_loglikelyhood.append(action_probability((beta_b[s]*v+eb+pb),action[t]))
                    pe = reward[t] - v[action[t]]
                    Kgain = np.square(sig[action[t]])/(np.square(sig[action[t]])+np.square(sig_o))
                    v[action[t]] = v[action[t]] + alpha[s] * pe
                    sig[action[t]] = np.sqrt((1-Kgain)*np.square(sig[action[t]]))
                else:
                    sample_loglikelyhood.append(0)
                sig = np.sqrt(np.square(sig)+np.square(sig_d))
            subject_loglikelyhood.append(sample_loglikelyhood)
        subject_loglikelyhood = np.array(subject_loglikelyhood)
        loo,_,_ = psisloo(subject_loglikelyhood)
        subject_loo.append(loo)
    return subject_loo

def loo_sup_dual_delta(file_name):
    beta_b_mean,phi_b_mean,persev_b_mean,pos_alpha_mean,neg_alpha_mean,beta_b_sd,phi_b_sd,persev_b_sd,pos_alpha_sd,neg_alpha_sd = read_parameter_sup_dual_sample(file_name)
    subjects = 60
    trials = 150
    subject_loo = []
    sample_num = 100
    
    for i in range(subjects):
        beta_b = np.random.normal(beta_b_mean[i], beta_b_sd[i], size=sample_num)
        phi_b = np.random.normal(phi_b_mean[i], phi_b_sd[i], size=sample_num)
        persev_b = np.random.normal(persev_b_mean[i], persev_b_sd[i], size=sample_num)
        pos_alpha = np.random.normal(pos_alpha_mean[i], pos_alpha_sd[i], size=sample_num)
        neg_alpha = np.random.normal(neg_alpha_mean[i], neg_alpha_sd[i], size=sample_num)
        
        action,reward,reward_B,_ =read_data(i) 
        comparision = rewards2comparison_sqrt(action,reward,reward_B,0)
        
        subject_loglikelyhood = []
    
        for s in range(sample_num):

            sample_loglikelyhood = []
            sig_o = 4
            sig_d = 2.8
            v = np.ones((4))*50
            sig = np.ones((4))*4
            for t in range(trials):
                if action[t]!=4:
                    #beta:temperature; v:action value; eb:exploration bonus; pb: perseverance bonus; re: random exploration
                    min_sig = np.min(sig)*np.ones((4))
                    re_sig = sig-min_sig
                    eb = phi_b[s] * re_sig
                    pb = np.zeros((4))
                    if t>0:
                        if action[t-1]!=4:
                            pb[action[t-1]] = persev_b[s]
                    # elife: beta[s]*(v+eb+pb+re); nhb: (beta[s]*v+eb+pb+re)
                    sample_loglikelyhood.append(action_probability((beta_b[s]*v+eb+pb),action[t]))
                    pe = reward[t] - v[action[t]]
                    Kgain = np.square(sig[action[t]])/(np.square(sig[action[t]])+np.square(sig_o))
                    if pe>0:
                        v[action[t]] = v[action[t]] + pos_alpha[s] * pe
                    else:
                        v[action[t]] = v[action[t]] + neg_alpha[s] * pe
                    sig[action[t]] = np.sqrt((1-Kgain)*np.square(sig[action[t]]))
                else:
                    sample_loglikelyhood.append(0)
                sig = np.sqrt(np.square(sig)+np.square(sig_d))
            subject_loglikelyhood.append(sample_loglikelyhood)
        subject_loglikelyhood = np.array(subject_loglikelyhood)
        loo,_,_ = psisloo(subject_loglikelyhood)
        subject_loo.append(loo)
    return subject_loo

def loo_sup_gamma(file_name):#nhb
    beta_b_mean,phi_b_mean,persev_b_mean,gamma_b_mean,beta_b_sd,phi_b_sd,persev_b_sd,gamma_b_sd = read_parameter_sup_sample(file_name)
    subjects = 60
    trials = 150
    subject_loo = []
    sample_num = 100
    
    for i in range(subjects):
        beta_b = np.random.normal(beta_b_mean[i], beta_b_sd[i], size=sample_num)
        phi_b = np.random.normal(phi_b_mean[i], phi_b_sd[i], size=sample_num)
        persev_b = np.random.normal(persev_b_mean[i], persev_b_sd[i], size=sample_num)
        gamma_b = np.random.normal(gamma_b_mean[i], gamma_b_sd[i], size=sample_num)
        
        action,reward,reward_B,_ =read_data(i) 
        comparision = rewards2comparison_sqrt(action,reward,reward_B,0)
        
        subject_loglikelyhood = []
    
        for s in range(sample_num):

            sample_loglikelyhood = []
            sig_o = 4
            sig_d = 2.8
            v = np.ones((4))*50
            sig = np.ones((4))*4
            for t in range(trials):
                if action[t]!=4:
                    #beta:temperature; v:action value; eb:exploration bonus; pb: perseverance bonus; re: random exploration
                    min_sig = np.min(sig)*np.ones((4))
                    re_sig = sig-min_sig
                    eb = phi_b[s] * re_sig
                    pb = np.zeros((4))
                    re = gamma_b[s] * v / np.sum(sig)
                    if t>0:
                        if action[t-1]!=4:
                            pb[action[t-1]] = persev_b[s]
                    # elife: beta[s]*(v+eb+pb+re); nhb: (beta[s]*v+eb+pb+re)
                    sample_loglikelyhood.append(action_probability((beta_b[s]*v+eb+pb+re),action[t]))
                    pe = reward[t] - v[action[t]]
                    Kgain = np.square(sig[action[t]])/(np.square(sig[action[t]])+np.square(sig_o))
                    v[action[t]] = v[action[t]] + Kgain * pe
                    sig[action[t]] = np.sqrt((1-Kgain)*np.square(sig[action[t]]))
                else:
                    sample_loglikelyhood.append(0)
                sig = np.sqrt(np.square(sig)+np.square(sig_d))
            subject_loglikelyhood.append(sample_loglikelyhood)
        subject_loglikelyhood = np.array(subject_loglikelyhood)
        loo,_,_ = psisloo(subject_loglikelyhood)
        subject_loo.append(loo)
    return subject_loo

def loo_sup_elife(file_name):#nhb
    beta_b_mean,phi_b_mean,persev_b_mean,beta_b_sd,phi_b_sd,persev_b_sd = read_parameter_no_gamma_sample(file_name)
    subjects = 60
    trials = 150
    subject_loo = []
    sample_num = 100
    
    for i in range(subjects):
        beta_b = np.random.normal(beta_b_mean[i], beta_b_sd[i], size=sample_num)
        phi_b = np.random.normal(phi_b_mean[i], phi_b_sd[i], size=sample_num)
        persev_b = np.random.normal(persev_b_mean[i], persev_b_sd[i], size=sample_num)
        
        action,reward,reward_B,_ =read_data(i) 
        comparision = rewards2comparison_sqrt(action,reward,reward_B,0)
        
        subject_loglikelyhood = []
    
        for s in range(sample_num):

            sample_loglikelyhood = []
            sig_o = 4
            sig_d = 2.8
            v = np.ones((4))*50
            sig = np.ones((4))*4
            for t in range(trials):
                if action[t]!=4:
                    #beta:temperature; v:action value; eb:exploration bonus; pb: perseverance bonus; re: random exploration
                    min_sig = np.min(sig)*np.ones((4))
                    re_sig = sig-min_sig
                    eb = phi_b[s] * re_sig
                    pb = np.zeros((4))
                    if t>0:
                        if action[t-1]!=4:
                            pb[action[t-1]] = persev_b[s]
                    # elife: beta[s]*(v+eb+pb+re); nhb: (beta[s]*v+eb+pb+re)
                    sample_loglikelyhood.append(action_probability(beta_b[s]*(v+eb+pb),action[t]))
                    pe = reward[t] - v[action[t]]
                    Kgain = np.square(sig[action[t]])/(np.square(sig[action[t]])+np.square(sig_o))
                    v[action[t]] = v[action[t]] + Kgain * pe
                    sig[action[t]] = np.sqrt((1-Kgain)*np.square(sig[action[t]]))
                else:
                    sample_loglikelyhood.append(0)
                sig = np.sqrt(np.square(sig)+np.square(sig_d))
            subject_loglikelyhood.append(sample_loglikelyhood)
        subject_loglikelyhood = np.array(subject_loglikelyhood)
        loo,_,_ = psisloo(subject_loglikelyhood)
        subject_loo.append(loo)
    return subject_loo

def loo_sup_competition_action(file_name):#nhb
    beta_b_mean,phi_b_mean,persev_b_mean,level_mean,beta_b_sd,phi_b_sd,persev_b_sd,level_sd = read_parameter_competition_sample(file_name)
    subjects = 60
    trials = 150
    subject_loo = []
    sample_num = 100
    
    for i in range(subjects):
        beta_b = np.random.normal(beta_b_mean[i], beta_b_sd[i], size=sample_num)
        phi_b = np.random.normal(phi_b_mean[i], phi_b_sd[i], size=sample_num)
        persev_b = np.random.normal(persev_b_mean[i], persev_b_sd[i], size=sample_num)
        level = np.random.normal(level_mean[i], level_sd[i], size=sample_num)
        
        action,reward,reward_B,_ =read_data(i) 
        comparision = rewards2comparison_sqrt(action,reward,reward_B,0)
        
        subject_loglikelyhood = []
    
        for s in range(sample_num):

            sample_loglikelyhood = []
            sig_o = 4
            sig_d = 2.8
            v = np.ones((4))*50
            sig = np.ones((4))*4
            for t in range(trials):
                if action[t]!=4:
                    #beta:temperature; v:action value; eb:exploration bonus; pb: perseverance bonus; re: random exploration
                    min_sig = np.min(sig)*np.ones((4))
                    re_sig = sig-min_sig
                    eb = (phi_b[s]) * re_sig
                    pb = np.zeros((4))
                    compare = np.zeros((4))
                    if t>0:
                        if action[t-1]!=4:
                            pb[action[t-1]] = (persev_b[s])
                            compare[action[t-1]] = comparision[t]
                    # elife: beta[s]*(v+eb+pb+re); nhb: (beta[s]*v+eb+pb+re)
                    
                    happy = v + level[s] * compare
                    sample_loglikelyhood.append(action_probability(((beta_b[s])*happy+eb+pb),action[t]))
                    pe = reward[t] - v[action[t]]
                    Kgain = np.square(sig[action[t]])/(np.square(sig[action[t]])+np.square(sig_o))
                    v[action[t]] = v[action[t]] + Kgain * pe
                    sig[action[t]] = np.sqrt((1-Kgain)*np.square(sig[action[t]]))
                else:
                    sample_loglikelyhood.append(0)
                sig = np.sqrt(np.square(sig)+np.square(sig_d))
            subject_loglikelyhood.append(sample_loglikelyhood)
        subject_loglikelyhood = np.array(subject_loglikelyhood)
        loo,_,_ = psisloo(subject_loglikelyhood)
        subject_loo.append(loo)
    return subject_loo

def loo_sup_competition_learn(file_name):#nhb
    beta_b_mean,phi_b_mean,persev_b_mean,level_mean,beta_b_sd,phi_b_sd,persev_b_sd,level_sd = read_parameter_competition_sample(file_name)
    subjects = 60
    trials = 150
    subject_loo = []
    sample_num = 100
    
    for i in range(subjects):
        beta_b = np.random.normal(beta_b_mean[i], beta_b_sd[i], size=sample_num)
        phi_b = np.random.normal(phi_b_mean[i], phi_b_sd[i], size=sample_num)
        persev_b = np.random.normal(persev_b_mean[i], persev_b_sd[i], size=sample_num)
        level = np.random.normal(level_mean[i], level_sd[i], size=sample_num)
        
        action,reward,reward_B,_ =read_data(i) 
        comparision = rewards2comparison_sqrt(action,reward,reward_B,0)
        
        subject_loglikelyhood = []
    
        for s in range(sample_num):

            sample_loglikelyhood = []
            sig_o = 4
            sig_d = 2.8
            v = np.ones((4))*50
            sig = np.ones((4))*4
            for t in range(trials):
                if action[t]!=4:
                    #beta:temperature; v:action value; eb:exploration bonus; pb: perseverance bonus; re: random exploration
                    min_sig = np.min(sig)*np.ones((4))
                    re_sig = sig-min_sig
                    eb = (phi_b[s]) * re_sig
                    pb = np.zeros((4))
                    if t>0:
                        if action[t-1]!=4:
                            pb[action[t-1]] = (persev_b[s])
                    # elife: beta[s]*(v+eb+pb+re); nhb: (beta[s]*v+eb+pb+re)
                    
                    sample_loglikelyhood.append(action_probability(((beta_b[s])*v+eb+pb),action[t]))
                    pe = reward[t] + level[s]*comparision[t] - v[action[t]]
                    Kgain = np.square(sig[action[t]])/(np.square(sig[action[t]])+np.square(sig_o))
                    v[action[t]] = v[action[t]] + Kgain * pe
                    sig[action[t]] = np.sqrt((1-Kgain)*np.square(sig[action[t]]))
                else:
                    sample_loglikelyhood.append(0)
                sig = np.sqrt(np.square(sig)+np.square(sig_d))
            subject_loglikelyhood.append(sample_loglikelyhood)
        subject_loglikelyhood = np.array(subject_loglikelyhood)
        loo,_,_ = psisloo(subject_loglikelyhood)
        subject_loo.append(loo)
    return subject_loo

def loo_sup_glm2_vep(file_name):#nhb
    beta_a_1_mean,phi_a_1_mean,persev_a_1_mean,beta_a_0_mean,phi_a_0_mean,persev_a_0_mean,beta_b_mean,phi_b_mean,persev_b_mean,beta_a_1_sd,phi_a_1_sd,persev_a_1_sd,beta_a_0_sd,phi_a_0_sd,persev_a_0_sd,beta_b_sd,phi_b_sd,persev_b_sd = read_parameter_glm2_no_gamma_sample(file_name)
    subjects = 60
    trials = 150
    subject_loo = []
    sample_num = 100
    
    
    for i in range(subjects):
        beta_a_0 = np.random.normal(beta_a_0_mean[i], beta_a_0_sd[i], size=sample_num)
        phi_a_0 = np.random.normal(phi_a_0_mean[i], phi_a_0_sd[i], size=sample_num)
        persev_a_0 = np.random.normal(persev_a_0_mean[i], persev_a_0_sd[i], size=sample_num)
        beta_a_1 = np.random.normal(beta_a_1_mean[i], beta_a_1_sd[i], size=sample_num)
        phi_a_1 = np.random.normal(phi_a_1_mean[i], phi_a_1_sd[i], size=sample_num)
        persev_a_1 = np.random.normal(persev_a_1_mean[i], persev_a_1_sd[i], size=sample_num)
        beta_b = np.random.normal(beta_b_mean[i], beta_b_sd[i], size=sample_num)
        phi_b = np.random.normal(phi_b_mean[i], phi_b_sd[i], size=sample_num)
        persev_b = np.random.normal(persev_b_mean[i], persev_b_sd[i], size=sample_num)
        
        action,reward,reward_B,_ =read_data(i) 
        comparision = rewards2comparison_sqrt(action,reward,reward_B,0)
        
        subject_loglikelyhood = []

        for s in range(sample_num):
            sample_loglikelyhood = []
            # loglikelyhood = 0
            sig_o = 4
            sig_d = 2.8
            v = np.ones((4))*50
            sig = np.ones((4))*4
            for t in range(trials):
                if action[t]!=4:
                    if comparision[t]<0:
                    
                        #beta:temperature; v:action value; eb:exploration bonus; pb: perseverance bonus; re: random exploration
                        min_sig = np.min(sig)*np.ones((4))
                        re_sig = sig-min_sig
                        eb = (phi_a_0[s]*comparision[t]+phi_b[s]) * re_sig
                        pb = np.zeros((4))
                        if t>0:
                            if action[t-1]!=4:
                                pb[action[t-1]] = (persev_a_0[s]*comparision[t]+persev_b[s])
                        # elife: beta[s]*(v+eb+pb+re); nhb: (beta[s]*v+eb+pb+re)
                        beta_t = beta_a_0[s]*comparision[t]+beta_b[s]
                        if beta_t<0:
                            beta_t = 0
                        sample_loglikelyhood.append(action_probability((beta_t*v+eb+pb),action[t]))
                    if comparision[t]>=0:
                        min_sig = np.min(sig)*np.ones((4))
                        re_sig = sig-min_sig
                        eb = (phi_a_1[s]*comparision[t]+phi_b[s]) * re_sig
                        pb = np.zeros((4))
                        if t>0:
                            if action[t-1]!=4:
                                pb[action[t-1]] = (persev_a_1[s]*comparision[t]+persev_b[s])
                        # elife: beta[s]*(v+eb+pb+re); nhb: (beta[s]*v+eb+pb+re)
                        beta_t = beta_a_1[s]*comparision[t]+beta_b[s]
                        if beta_t<0:
                            beta_t = 0
                        sample_loglikelyhood.append(action_probability((beta_t*v+eb+pb),action[t]))
                    pe = reward[t] - v[action[t]]
                    Kgain = np.square(sig[action[t]])/(np.square(sig[action[t]])+np.square(sig_o))
                    v[action[t]] = v[action[t]] + Kgain * pe
                    sig[action[t]] = np.sqrt((1-Kgain)*np.square(sig[action[t]]))
                else:
                    sample_loglikelyhood.append(0)
                sig = np.sqrt(np.square(sig)+np.square(sig_d))
            subject_loglikelyhood.append(sample_loglikelyhood)
        subject_loglikelyhood = np.array(subject_loglikelyhood)
        loo,_,_ = psisloo(subject_loglikelyhood)
        subject_loo.append(loo)
    return subject_loo

def loo_sup_glm2_v(file_name):#nhb
    beta_b_mean,phi_b_mean,persev_b_mean,beta_a_0_mean,beta_a_1_mean,beta_b_sd,phi_b_sd,persev_b_sd,beta_a_0_sd,beta_a_1_sd = read_parameter_glm2_no_gamma_beta_sample(file_name)
    subjects = 60
    trials = 150
    subject_loo = []
    sample_num = 100
    
    
    for i in range(subjects):
        beta_a_0 = np.random.normal(beta_a_0_mean[i], beta_a_0_sd[i], size=sample_num)

        beta_a_1 = np.random.normal(beta_a_1_mean[i], beta_a_1_sd[i], size=sample_num)

        beta_b = np.random.normal(beta_b_mean[i], beta_b_sd[i], size=sample_num)
        phi_b = np.random.normal(phi_b_mean[i], phi_b_sd[i], size=sample_num)
        persev_b = np.random.normal(persev_b_mean[i], persev_b_sd[i], size=sample_num)
        
        action,reward,reward_B,_ =read_data(i) 
        comparision = rewards2comparison_sqrt(action,reward,reward_B,0)
        
        subject_loglikelyhood = []

        for s in range(sample_num):
            sample_loglikelyhood = []
            # loglikelyhood = 0
            sig_o = 4
            sig_d = 2.8
            v = np.ones((4))*50
            sig = np.ones((4))*4
            for t in range(trials):
                if action[t]!=4:
                    if comparision[t]<0:
                        #beta:temperature; v:action value; eb:exploration bonus; pb: perseverance bonus; re: random exploration
                        min_sig = np.min(sig)*np.ones((4))
                        re_sig = sig-min_sig
                        eb = (phi_b[s]) * re_sig
                        pb = np.zeros((4))
                        if t>0:
                            if action[t-1]!=4:
                                pb[action[t-1]] = (persev_b[s])
                        # elife: beta[s]*(v+eb+pb+re); nhb: (beta[s]*v+eb+pb+re)
                        beta_t = beta_a_0[s]*comparision[t]+beta_b[s]
                        if beta_t<0:
                            beta_t = 0
                        sample_loglikelyhood.append(action_probability((beta_t*v+eb+pb),action[t]))
                    if comparision[t]>=0:
                        min_sig = np.min(sig)*np.ones((4))
                        re_sig = sig-min_sig
                        eb = (phi_b[s]) * re_sig
                        pb = np.zeros((4))
                        if t>0:
                            if action[t-1]!=4:
                                pb[action[t-1]] = (persev_b[s])
                        # elife: beta[s]*(v+eb+pb+re); nhb: (beta[s]*v+eb+pb+re)
                        beta_t = beta_a_1[s]*comparision[t]+beta_b[s]
                        if beta_t<0:
                            beta_t = 0
                        sample_loglikelyhood.append(action_probability((beta_t*v+eb+pb),action[t]))
                    pe = reward[t] - v[action[t]]
                    Kgain = np.square(sig[action[t]])/(np.square(sig[action[t]])+np.square(sig_o))
                    v[action[t]] = v[action[t]] + Kgain * pe
                    sig[action[t]] = np.sqrt((1-Kgain)*np.square(sig[action[t]]))
                else:
                    sample_loglikelyhood.append(0)
                sig = np.sqrt(np.square(sig)+np.square(sig_d))
            subject_loglikelyhood.append(sample_loglikelyhood)
        subject_loglikelyhood = np.array(subject_loglikelyhood)
        loo,_,_ = psisloo(subject_loglikelyhood)
        subject_loo.append(loo)
    return subject_loo

def loo_sup_glm2_e(file_name):#nhb
    beta_b_mean,phi_b_mean,persev_b_mean,phi_a_0_mean,phi_a_1_mean,beta_b_sd,phi_b_sd,persev_b_sd,phi_a_0_sd,phi_a_1_sd = read_parameter_glm2_no_gamma_beta_sample(file_name)
    subjects = 60
    trials = 150
    subject_loo = []
    sample_num = 100
    
    
    for i in range(subjects):
        phi_a_0 = np.random.normal(phi_a_0_mean[i], phi_a_0_sd[i], size=sample_num)

        phi_a_1 = np.random.normal(phi_a_1_mean[i], phi_a_1_sd[i], size=sample_num)

        beta_b = np.random.normal(beta_b_mean[i], beta_b_sd[i], size=sample_num)
        phi_b = np.random.normal(phi_b_mean[i], phi_b_sd[i], size=sample_num)
        persev_b = np.random.normal(persev_b_mean[i], persev_b_sd[i], size=sample_num)
        
        action,reward,reward_B,_ =read_data(i) 
        comparision = rewards2comparison_sqrt(action,reward,reward_B,0)
        
        subject_loglikelyhood = []

        for s in range(sample_num):
            sample_loglikelyhood = []
            # loglikelyhood = 0
            sig_o = 4
            sig_d = 2.8
            v = np.ones((4))*50
            sig = np.ones((4))*4
            for t in range(trials):
                if action[t]!=4:
                    if comparision[t]<0:
                        
                        #beta:temperature; v:action value; eb:exploration bonus; pb: perseverance bonus; re: random exploration
                        min_sig = np.min(sig)*np.ones((4))
                        re_sig = sig-min_sig
                        eb = (phi_a_0[s]*comparision[t]+phi_b[s]) * re_sig
                        pb = np.zeros((4))
                        if t>0:
                            if action[t-1]!=4:
                                pb[action[t-1]] = (persev_b[s])
                        # elife: beta[s]*(v+eb+pb+re); nhb: (beta[s]*v+eb+pb+re)
                        sample_loglikelyhood.append(action_probability(((beta_b[s])*v+eb+pb),action[t]))
                    if comparision[t]>=0:
                        min_sig = np.min(sig)*np.ones((4))
                        re_sig = sig-min_sig
                        eb = (phi_a_1[s]*comparision[t]+phi_b[s]) * re_sig
                        pb = np.zeros((4))
                        if t>0:
                            if action[t-1]!=4:
                                pb[action[t-1]] = (persev_b[s])
                        # elife: beta[s]*(v+eb+pb+re); nhb: (beta[s]*v+eb+pb+re)
                        sample_loglikelyhood.append(action_probability(((beta_b[s])*v+eb+pb),action[t]))
                    pe = reward[t] - v[action[t]]
                    Kgain = np.square(sig[action[t]])/(np.square(sig[action[t]])+np.square(sig_o))
                    v[action[t]] = v[action[t]] + Kgain * pe
                    sig[action[t]] = np.sqrt((1-Kgain)*np.square(sig[action[t]]))
                else:
                    sample_loglikelyhood.append(0)
                sig = np.sqrt(np.square(sig)+np.square(sig_d))
            subject_loglikelyhood.append(sample_loglikelyhood)
        subject_loglikelyhood = np.array(subject_loglikelyhood)
        loo,_,_ = psisloo(subject_loglikelyhood)
        subject_loo.append(loo)
    return subject_loo

def loo_sup_glm2_p(file_name):#nhb
    beta_b_mean,phi_b_mean,persev_b_mean,persev_a_0_mean,persev_a_1_mean,beta_b_sd,phi_b_sd,persev_b_sd,persev_a_0_sd,persev_a_1_sd = read_parameter_glm2_no_gamma_beta_sample(file_name)
    subjects = 60
    trials = 150
    subject_loo = []
    sample_num = 100
    
    
    for i in range(subjects):
        persev_a_0 = np.random.normal(persev_a_0_mean[i], persev_a_0_sd[i], size=sample_num)

        persev_a_1 = np.random.normal(persev_a_1_mean[i], persev_a_1_sd[i], size=sample_num)

        beta_b = np.random.normal(beta_b_mean[i], beta_b_sd[i], size=sample_num)
        phi_b = np.random.normal(phi_b_mean[i], phi_b_sd[i], size=sample_num)
        persev_b = np.random.normal(persev_b_mean[i], persev_b_sd[i], size=sample_num)
        
        action,reward,reward_B,_ =read_data(i) 
        comparision = rewards2comparison_sqrt(action,reward,reward_B,0)
        
        subject_loglikelyhood = []

        for s in range(sample_num):
            sample_loglikelyhood = []
            # loglikelyhood = 0
            sig_o = 4
            sig_d = 2.8
            v = np.ones((4))*50
            sig = np.ones((4))*4
            for t in range(trials):
                if action[t]!=4:
                    if comparision[t]<0:
                        
                        #beta:temperature; v:action value; eb:exploration bonus; pb: perseverance bonus; re: random exploration
                        min_sig = np.min(sig)*np.ones((4))
                        re_sig = sig-min_sig
                        eb = (phi_b[s]) * re_sig
                        pb = np.zeros((4))
                        if t>0:
                            if action[t-1]!=4:
                                pb[action[t-1]] = (persev_a_0[s]*comparision[t]+persev_b[s])
                        # elife: beta[s]*(v+eb+pb+re); nhb: (beta[s]*v+eb+pb+re)
                        sample_loglikelyhood.append(action_probability(((beta_b[s])*v+eb+pb),action[t]))
                    if comparision[t]>=0:
                        min_sig = np.min(sig)*np.ones((4))
                        re_sig = sig-min_sig
                        eb = (phi_b[s]) * re_sig
                        pb = np.zeros((4))
                        if t>0:
                            if action[t-1]!=4:
                                pb[action[t-1]] = (persev_a_1[s]*comparision[t]+persev_b[s])
                        # elife: beta[s]*(v+eb+pb+re); nhb: (beta[s]*v+eb+pb+re)
                        sample_loglikelyhood.append(action_probability(((beta_b[s])*v+eb+pb),action[t]))
                
                    pe = reward[t] - v[action[t]]
                    Kgain = np.square(sig[action[t]])/(np.square(sig[action[t]])+np.square(sig_o))
                    v[action[t]] = v[action[t]] + Kgain * pe
                    sig[action[t]] = np.sqrt((1-Kgain)*np.square(sig[action[t]]))
                else:
                    sample_loglikelyhood.append(0)
                sig = np.sqrt(np.square(sig)+np.square(sig_d))
            subject_loglikelyhood.append(sample_loglikelyhood)
        subject_loglikelyhood = np.array(subject_loglikelyhood)
        loo,_,_ = psisloo(subject_loglikelyhood)
        subject_loo.append(loo)
    return subject_loo

def loo_glm_absolute(file_name):#nhb
    beta_a_mean,phi_a_mean,persev_a_mean,beta_b_mean,phi_b_mean,persev_b_mean,beta_a_sd,phi_a_sd,persev_a_sd,beta_b_sd,phi_b_sd,persev_b_sd = read_parameter_glm_3_para_sample(file_name)
    subjects = 60
    trials = 150
    subject_loo = []
    sample_num = 100
    
    
    for i in range(subjects):
        beta_a = np.random.normal(beta_a_mean[i], beta_a_sd[i], size=sample_num)
        phi_a = np.random.normal(phi_a_mean[i], phi_a_sd[i], size=sample_num)
        persev_a = np.random.normal(persev_a_mean[i], persev_a_sd[i], size=sample_num)
        beta_b = np.random.normal(beta_b_mean[i], beta_b_sd[i], size=sample_num)
        phi_b = np.random.normal(phi_b_mean[i], phi_b_sd[i], size=sample_num)
        persev_b = np.random.normal(persev_b_mean[i], persev_b_sd[i], size=sample_num)
        
        action,reward,reward_B,_ =read_data(i) 
        comparision = rewards2comparison_sqrt(action,reward,reward_B,0)
        subject_loglikelyhood = []

        for s in range(sample_num):
            sample_loglikelyhood = []
            # loglikelyhood = 0
            sig_o = 4
            sig_d = 2.8
            v = np.ones((4))*50
            sig = np.ones((4))*4
            for t in range(trials):
                if action[t]!=4:
                    #beta:temperature; v:action value; eb:exploration bonus; pb: perseverance bonus; re: random exploration
                    min_sig = np.min(sig)*np.ones((4))
                    re_sig = sig-min_sig
                    eb = (phi_a[s]*np.abs(comparision[t])+phi_b[s]) * re_sig
                    pb = np.zeros((4))
                    if t>0:
                        if action[t-1]!=4:
                            pb[action[t-1]] = (persev_a[s]*np.abs(comparision[t])+persev_b[s])
                    # elife: beta[s]*(v+eb+pb+re); nhb: (beta[s]*v+eb+pb+re)
                    beta_t = beta_a[s]*np.abs(comparision[t])+beta_b[s]
                    if beta_t<0:
                        beta_t = 0
                    sample_loglikelyhood.append(action_probability((beta_t*v+eb+pb),action[t]))
                    pe = reward[t] - v[action[t]]
                    Kgain = np.square(sig[action[t]])/(np.square(sig[action[t]])+np.square(sig_o))
                    v[action[t]] = v[action[t]] + Kgain * pe
                    sig[action[t]] = np.sqrt((1-Kgain)*np.square(sig[action[t]]))
                else:
                    sample_loglikelyhood.append(0)
                sig = np.sqrt(np.square(sig)+np.square(sig_d))
            subject_loglikelyhood.append(sample_loglikelyhood)
        subject_loglikelyhood = np.array(subject_loglikelyhood)
        loo,_,_ = psisloo(subject_loglikelyhood)
        subject_loo.append(loo)
    return subject_loo

def loo_persev_trace(file_name):#nhb
    beta_a_mean,phi_a_mean,persev_a_mean,beta_b_mean,phi_b_mean,persev_b_mean,tau_mean,beta_a_sd,phi_a_sd,persev_a_sd,beta_b_sd,phi_b_sd,persev_b_sd,tau_sd = read_parameter_persev_trace_sample(file_name)
    subjects = 60
    trials = 150
    subject_loo = []
    sample_num = 100
    for i in range(subjects):
        beta_a = np.random.normal(beta_a_mean[i], beta_a_sd[i], size=sample_num)
        phi_a = np.random.normal(phi_a_mean[i], phi_a_sd[i], size=sample_num)
        persev_a = np.random.normal(persev_a_mean[i], persev_a_sd[i], size=sample_num)
        beta_b = np.random.normal(beta_b_mean[i], beta_b_sd[i], size=sample_num)
        phi_b = np.random.normal(phi_b_mean[i], phi_b_sd[i], size=sample_num)
        persev_b = np.random.normal(persev_b_mean[i], persev_b_sd[i], size=sample_num)
        tau = np.random.normal(tau_mean[i], tau_sd[i], size=sample_num)
        
        action,reward,reward_B,_ =read_data(i) 
        comparision = rewards2comparison_sqrt(action,reward,reward_B,0)
        
        subject_loglikelyhood = []

        for s in range(sample_num):
            sample_loglikelyhood = []
            # loglikelyhood = 0
            sig_o = 4
            sig_d = 2.8
            v = np.ones((4))*50
            sig = np.ones((4))*4
            pb_last = np.zeros((4))
            for t in range(trials):
                if action[t]!=4:
                    #beta:temperature; v:action value; eb:exploration bonus; pb: perseverance bonus; re: random exploration
                    min_sig = np.min(sig)*np.ones((4))
                    re_sig = sig-min_sig
                    eb = (phi_a[s]*comparision[t]+phi_b[s]) * re_sig
                    pb = np.zeros((4))
                    if t>0:
                        if action[t-1]!=4:
                            pb[action[t-1]] = (persev_a[s]*comparision[t]+persev_b[s])
                    pb = pb + tau[s]*pb_last
                    pb_last = pb
                    # elife: beta[s]*(v+eb+pb+re); nhb: (beta[s]*v+eb+pb+re)
                    beta_t = beta_a[s]*comparision[t]+beta_b[s]
                    if beta_t<0:
                        beta_t = 0
                    sample_loglikelyhood.append(action_probability((beta_t*v+eb+pb),action[t]))
                    pe = reward[t] - v[action[t]]
                    Kgain = np.square(sig[action[t]])/(np.square(sig[action[t]])+np.square(sig_o))
                    v[action[t]] = v[action[t]] + Kgain * pe
                    sig[action[t]] = np.sqrt((1-Kgain)*np.square(sig[action[t]]))
                else:
                    sample_loglikelyhood.append(0)
                sig = np.sqrt(np.square(sig)+np.square(sig_d))
            subject_loglikelyhood.append(sample_loglikelyhood)
        subject_loglikelyhood = np.array(subject_loglikelyhood)
        loo,_,_ = psisloo(subject_loglikelyhood)
        subject_loo.append(loo)
    return subject_loo

def loo_persev_trace_glm_p(file_name):#nhb
    persev_a_mean,beta_b_mean,phi_b_mean,persev_b_mean,tau_mean,persev_a_sd,beta_b_sd,phi_b_sd,persev_b_sd,tau_sd = read_parameter_persev_trace_glm_p_sample(file_name)
    subjects = 60
    trials = 150
    subject_loo = []
    sample_num = 100
    for i in range(subjects):
        persev_a = np.random.normal(persev_a_mean[i], persev_a_sd[i], size=sample_num)
        beta_b = np.random.normal(beta_b_mean[i], beta_b_sd[i], size=sample_num)
        phi_b = np.random.normal(phi_b_mean[i], phi_b_sd[i], size=sample_num)
        persev_b = np.random.normal(persev_b_mean[i], persev_b_sd[i], size=sample_num)
        tau = np.random.normal(tau_mean[i], tau_sd[i], size=sample_num)
        
        action,reward,reward_B,_ =read_data(i) 
        comparision = rewards2comparison_sqrt(action,reward,reward_B,0)
        
        subject_loglikelyhood = []

        for s in range(sample_num):
            sample_loglikelyhood = []
            # loglikelyhood = 0
            sig_o = 4
            sig_d = 2.8
            v = np.ones((4))*50
            sig = np.ones((4))*4
            pb_last = np.zeros((4))
            for t in range(trials):
                if action[t]!=4:
                    #beta:temperature; v:action value; eb:exploration bonus; pb: perseverance bonus; re: random exploration
                    min_sig = np.min(sig)*np.ones((4))
                    re_sig = sig-min_sig
                    eb = phi_b[s] * re_sig
                    pb = np.zeros((4))
                    if t>0:
                        if action[t-1]!=4:
                            pb[action[t-1]] = (persev_a[s]*comparision[t]+persev_b[s])
                    pb = pb + tau[s]*pb_last
                    pb_last = pb
                    # elife: beta[s]*(v+eb+pb+re); nhb: (beta[s]*v+eb+pb+re)
                    sample_loglikelyhood.append(action_probability((beta_b[s]*v+eb+pb),action[t]))
                    pe = reward[t] - v[action[t]]
                    Kgain = np.square(sig[action[t]])/(np.square(sig[action[t]])+np.square(sig_o))
                    v[action[t]] = v[action[t]] + Kgain * pe
                    sig[action[t]] = np.sqrt((1-Kgain)*np.square(sig[action[t]]))
                else:
                    sample_loglikelyhood.append(0)
                sig = np.sqrt(np.square(sig)+np.square(sig_d))
            subject_loglikelyhood.append(sample_loglikelyhood)
        subject_loglikelyhood = np.array(subject_loglikelyhood)
        loo,_,_ = psisloo(subject_loglikelyhood)
        subject_loo.append(loo)
    return subject_loo

def loo_persev_trace_vep(file_name):#nhb
    beta_b_mean,phi_b_mean,persev_b_mean,tau_mean,beta_b_sd,phi_b_sd,persev_b_sd,tau_sd = read_parameter_persev_trace_vep_sample(file_name)
    subjects = 60
    trials = 150
    subject_loo = []
    sample_num = 100
    for i in range(subjects):
        beta_b = np.random.normal(beta_b_mean[i], beta_b_sd[i], size=sample_num)
        phi_b = np.random.normal(phi_b_mean[i], phi_b_sd[i], size=sample_num)
        persev_b = np.random.normal(persev_b_mean[i], persev_b_sd[i], size=sample_num)
        tau = np.random.normal(tau_mean[i], tau_sd[i], size=sample_num)
        
        action,reward,reward_B,_ =read_data(i) 
        comparision = rewards2comparison_sqrt(action,reward,reward_B,0)
        
        subject_loglikelyhood = []

        for s in range(sample_num):
            sample_loglikelyhood = []
            # loglikelyhood = 0
            sig_o = 4
            sig_d = 2.8
            v = np.ones((4))*50
            sig = np.ones((4))*4
            pb_last = np.zeros((4))
            for t in range(trials):
                if action[t]!=4:
                    #beta:temperature; v:action value; eb:exploration bonus; pb: perseverance bonus; re: random exploration
                    min_sig = np.min(sig)*np.ones((4))
                    re_sig = sig-min_sig
                    eb = phi_b[s] * re_sig
                    pb = np.zeros((4))
                    if t>0:
                        if action[t-1]!=4:
                            pb[action[t-1]] = persev_b[s]
                    pb = pb + tau[s]*pb_last
                    pb_last = pb
                    # elife: beta[s]*(v+eb+pb+re); nhb: (beta[s]*v+eb+pb+re)
                    sample_loglikelyhood.append(action_probability((beta_b[s]*v+eb+pb),action[t]))
                    pe = reward[t] - v[action[t]]
                    Kgain = np.square(sig[action[t]])/(np.square(sig[action[t]])+np.square(sig_o))
                    v[action[t]] = v[action[t]] + Kgain * pe
                    sig[action[t]] = np.sqrt((1-Kgain)*np.square(sig[action[t]]))
                else:
                    sample_loglikelyhood.append(0)
                sig = np.sqrt(np.square(sig)+np.square(sig_d))
            subject_loglikelyhood.append(sample_loglikelyhood)
        subject_loglikelyhood = np.array(subject_loglikelyhood)
        loo,_,_ = psisloo(subject_loglikelyhood)
        subject_loo.append(loo)
    return subject_loo

In [30]:
def log_likelyhood_nhb_glm_ru_no_gamma(file_name):#nhb
    beta_a,phi_a,persev_a,beta_b,phi_b,persev_b = read_parameter_glm_3_para(file_name)
    subjects = 60
    trials = 150
    subject_loglikelyhood = []
    for s in range(subjects):
        action,reward,reward_B,_ =read_data(s) 
        comparision = rewards2comparison(action,reward,reward_B,0)
        loglikelyhood = 0
        sig_o = 4
        sig_d = 2.8
        v = np.ones((4))*50
        sig = np.ones((4))*4
        for t in range(trials):
            if action[t]!=4:
                #beta:temperature; v:action value; eb:exploration bonus; pb: perseverance bonus; re: random exploration
                min_sig = np.min(sig)*np.ones((4))
                re_sig = sig-min_sig
                eb = (phi_a[s]*comparision[t]+phi_b[s]) * re_sig
                pb = np.zeros((4))
                if t>0:
                    if action[t-1]!=4:
                        pb[action[t-1]] = (persev_a[s]*comparision[t]+persev_b[s])
                # elife: beta[s]*(v+eb+pb+re); nhb: (beta[s]*v+eb+pb+re)
                beta_t = beta_a[s]*comparision[t]+beta_b[s]
                if beta_t < 0:
                    beta_t = 0
                loglikelyhood += action_probability((beta_t*v+eb+pb),action[t])
                pe = reward[t] - v[action[t]]
                Kgain = np.square(sig[action[t]])/(np.square(sig[action[t]])+np.square(sig_o))
                v[action[t]] = v[action[t]] + Kgain * pe
                sig[action[t]] = np.sqrt((1-Kgain)*np.square(sig[action[t]]))
            sig = np.sqrt(np.square(sig)+np.square(sig_d))
        subject_loglikelyhood.append(loglikelyhood)
    return subject_loglikelyhood

def log_likelyhood_nhb_glm_ru_no_gamma_beta_sqrt(file_name):#nhb
    beta_a,beta_b,phi_b,persev_b = read_parameter_glm_no_gamma_para(file_name)
    subjects = 60
    trials = 150
    subject_loglikelyhood = []
    for s in range(subjects):
        action,reward,reward_B,_ =read_data(s) 
        comparision = rewards2comparison_sqrt(action,reward,reward_B,0)
        loglikelyhood = 0
        sig_o = 4
        sig_d = 2.8
        v = np.ones((4))*50
        sig = np.ones((4))*4
        for t in range(trials):
            if action[t]!=4:
                #beta:temperature; v:action value; eb:exploration bonus; pb: perseverance bonus; re: random exploration
                min_sig = np.min(sig)*np.ones((4))
                re_sig = sig-min_sig
                eb = phi_b[s] * re_sig
                pb = np.zeros((4))
                if t>0:
                    if action[t-1]!=4:
                        pb[action[t-1]] = persev_b[s]
                # elife: beta[s]*(v+eb+pb+re); nhb: (beta[s]*v+eb+pb+re)
                beta_t = beta_a[s]*comparision[t]+beta_b[s]
                if beta_t < 0:
                    beta_t = 0
                loglikelyhood += action_probability((beta_t*v+eb+pb),action[t])
                pe = reward[t] - v[action[t]]
                Kgain = np.square(sig[action[t]])/(np.square(sig[action[t]])+np.square(sig_o))
                v[action[t]] = v[action[t]] + Kgain * pe
                sig[action[t]] = np.sqrt((1-Kgain)*np.square(sig[action[t]]))
            sig = np.sqrt(np.square(sig)+np.square(sig_d))
        subject_loglikelyhood.append(loglikelyhood)
    return subject_loglikelyhood

def log_likelyhood_nhb_glm_ru_no_gamma_phi_sqrt(file_name):#nhb
    phi_a,beta_b,phi_b,persev_b = read_parameter_glm_no_gamma_para(file_name)
    subjects = 60
    trials = 150
    subject_loglikelyhood = []
    for s in range(subjects):
        action,reward,reward_B,_ =read_data(s) 
        comparision = rewards2comparison_sqrt(action,reward,reward_B,0)
        loglikelyhood = 0
        sig_o = 4
        sig_d = 2.8
        v = np.ones((4))*50
        sig = np.ones((4))*4
        for t in range(trials):
            if action[t]!=4:
                #beta:temperature; v:action value; eb:exploration bonus; pb: perseverance bonus; re: random exploration
                min_sig = np.min(sig)*np.ones((4))
                re_sig = sig-min_sig
                eb = (phi_a[s]*comparision[t]+phi_b[s]) * re_sig
                pb = np.zeros((4))
                if t>0:
                    if action[t-1]!=4:
                        pb[action[t-1]] = persev_b[s]
                # elife: beta[s]*(v+eb+pb+re); nhb: (beta[s]*v+eb+pb+re)
                loglikelyhood += action_probability((beta_b[s]*v+eb+pb),action[t])
                pe = reward[t] - v[action[t]]
                Kgain = np.square(sig[action[t]])/(np.square(sig[action[t]])+np.square(sig_o))
                v[action[t]] = v[action[t]] + Kgain * pe
                sig[action[t]] = np.sqrt((1-Kgain)*np.square(sig[action[t]]))
            sig = np.sqrt(np.square(sig)+np.square(sig_d))
        subject_loglikelyhood.append(loglikelyhood)
    return subject_loglikelyhood

def log_likelyhood_nhb_glm_ru_no_gamma_persev_sqrt(file_name):#nhb
    persev_a,beta_b,phi_b,persev_b = read_parameter_glm_no_gamma_para(file_name)
    subjects = 60
    trials = 150
    subject_loglikelyhood = []
    for s in range(subjects):
        action,reward,reward_B,_ =read_data(s) 
        comparision = rewards2comparison_sqrt(action,reward,reward_B,0)
        loglikelyhood = 0
        sig_o = 4
        sig_d = 2.8
        v = np.ones((4))*50
        sig = np.ones((4))*4
        for t in range(trials):
            if action[t]!=4:
                #beta:temperature; v:action value; eb:exploration bonus; pb: perseverance bonus; re: random exploration
                min_sig = np.min(sig)*np.ones((4))
                re_sig = sig-min_sig
                eb = phi_b[s] * re_sig
                pb = np.zeros((4))
                if t>0:
                    if action[t-1]!=4:
                        pb[action[t-1]] = (persev_a[s]*comparision[t]+persev_b[s])
                # elife: beta[s]*(v+eb+pb+re); nhb: (beta[s]*v+eb+pb+re)
                loglikelyhood += action_probability((beta_b[s]*v+eb+pb),action[t])
                pe = reward[t] - v[action[t]]
                Kgain = np.square(sig[action[t]])/(np.square(sig[action[t]])+np.square(sig_o))
                v[action[t]] = v[action[t]] + Kgain * pe
                sig[action[t]] = np.sqrt((1-Kgain)*np.square(sig[action[t]]))
            sig = np.sqrt(np.square(sig)+np.square(sig_d))
        subject_loglikelyhood.append(loglikelyhood)
    return subject_loglikelyhood


def log_likelyhood_glm_gamma(file_name):#nhb
    beta_a,phi_a,persev_a,gamma_a,beta_b,phi_b,persev_b,gamma_b = read_parameter_glm(file_name)
    subjects = 60
    trials = 150
    subject_loglikelyhood = []
    for s in range(subjects):
        action,reward,reward_B,_ =read_data(s) 
        comparision = rewards2comparison_sqrt(action,reward,reward_B,0)
        loglikelyhood = 0
        sig_o = 4
        sig_d = 2.8
        v = np.ones((4))*50
        sig = np.ones((4))*4
        for t in range(trials):
            if action[t]!=4:
                #beta:temperature; v:action value; eb:exploration bonus; pb: perseverance bonus; re: random exploration
                min_sig = np.min(sig)*np.ones((4))
                re_sig = sig-min_sig
                eb = (phi_a[s]*comparision[t]+phi_b[s]) * re_sig
                pb = np.zeros((4))
                re = (gamma_a[s]*comparision[t]+gamma_b[s])*v/np.sum(sig)
                if t>0:
                    if action[t-1]!=4:
                        pb[action[t-1]] = (persev_a[s]*comparision[t]+persev_b[s])
                # elife: beta[s]*(v+eb+pb+re); nhb: (beta[s]*v+eb+pb+re)
                beta_t = beta_a[s]*comparision[t]+beta_b[s]
                if beta_t < 0:
                    beta_t = 0
                loglikelyhood += action_probability((beta_t*v+eb+pb+re),action[t])
                pe = reward[t] - v[action[t]]
                Kgain = np.square(sig[action[t]])/(np.square(sig[action[t]])+np.square(sig_o))
                v[action[t]] = v[action[t]] + Kgain * pe
                sig[action[t]] = np.sqrt((1-Kgain)*np.square(sig[action[t]]))
            sig = np.sqrt(np.square(sig)+np.square(sig_d))
        subject_loglikelyhood.append(loglikelyhood)
    return subject_loglikelyhood

def log_likelyhood_glm_gamma_phi(file_name):#nhb
    phi_a,gamma_a,beta_b,phi_b,persev_b,gamma_b = read_parameter_glm_3_para(file_name)
    subjects = 60
    trials = 150
    subject_loglikelyhood = []
    for s in range(subjects):
        action,reward,reward_B,_ =read_data(s) 
        comparision = rewards2comparison_sqrt(action,reward,reward_B,0)
        loglikelyhood = 0
        sig_o = 4
        sig_d = 2.8
        v = np.ones((4))*50
        sig = np.ones((4))*4
        for t in range(trials):
            if action[t]!=4:
                #beta:temperature; v:action value; eb:exploration bonus; pb: perseverance bonus; re: random exploration
                min_sig = np.min(sig)*np.ones((4))
                re_sig = sig-min_sig
                eb = (phi_a[s]*comparision[t]+phi_b[s]) * re_sig
                pb = np.zeros((4))
                re = (gamma_a[s]*comparision[t]+gamma_b[s])*v/np.sum(sig)
                if t>0:
                    if action[t-1]!=4:
                        pb[action[t-1]] = (persev_b[s])
                # elife: beta[s]*(v+eb+pb+re); nhb: (beta[s]*v+eb+pb+re)
                loglikelyhood += action_probability(((beta_b[s])*v+eb+pb+re),action[t])
                pe = reward[t] - v[action[t]]
                Kgain = np.square(sig[action[t]])/(np.square(sig[action[t]])+np.square(sig_o))
                v[action[t]] = v[action[t]] + Kgain * pe
                sig[action[t]] = np.sqrt((1-Kgain)*np.square(sig[action[t]]))
            sig = np.sqrt(np.square(sig)+np.square(sig_d))
        subject_loglikelyhood.append(loglikelyhood)
    return subject_loglikelyhood


def log_likelyhood_glm_only_gamma(file_name):#nhb
    gamma_a,beta_b,phi_b,persev_b,gamma_b = read_parameter_glm_para(file_name)
    subjects = 60
    trials = 150
    subject_loglikelyhood = []
    for s in range(subjects):
        action,reward,reward_B,_ =read_data(s) 
        comparision = rewards2comparison_sqrt(action,reward,reward_B,0)
        loglikelyhood = 0
        sig_o = 4
        sig_d = 2.8
        v = np.ones((4))*50
        sig = np.ones((4))*4
        for t in range(trials):
            if action[t]!=4:
                #beta:temperature; v:action value; eb:exploration bonus; pb: perseverance bonus; re: random exploration
                min_sig = np.min(sig)*np.ones((4))
                re_sig = sig-min_sig
                eb = (phi_b[s]) * re_sig
                pb = np.zeros((4))
                re = (gamma_a[s]*comparision[t]+gamma_b[s])*v/np.sum(sig)
                if t>0:
                    if action[t-1]!=4:
                        pb[action[t-1]] = (persev_b[s])
                # elife: beta[s]*(v+eb+pb+re); nhb: (beta[s]*v+eb+pb+re)
                loglikelyhood += action_probability(((beta_b[s])*v+eb+pb+re),action[t])
                pe = reward[t] - v[action[t]]
                Kgain = np.square(sig[action[t]])/(np.square(sig[action[t]])+np.square(sig_o))
                v[action[t]] = v[action[t]] + Kgain * pe
                sig[action[t]] = np.sqrt((1-Kgain)*np.square(sig[action[t]]))
            sig = np.sqrt(np.square(sig)+np.square(sig_d))
        subject_loglikelyhood.append(loglikelyhood)
    return subject_loglikelyhood


def log_likelyhood_glm_only_phi(file_name):#nhb
    phi_a,beta_b,phi_b,persev_b,gamma_b = read_parameter_glm_para(file_name)
    subjects = 60
    trials = 150
    subject_loglikelyhood = []
    for s in range(subjects):
        action,reward,reward_B,_ =read_data(s) 
        comparision = rewards2comparison_sqrt(action,reward,reward_B,0)
        loglikelyhood = 0
        sig_o = 4
        sig_d = 2.8
        v = np.ones((4))*50
        sig = np.ones((4))*4
        for t in range(trials):
            if action[t]!=4:
                #beta:temperature; v:action value; eb:exploration bonus; pb: perseverance bonus; re: random exploration
                min_sig = np.min(sig)*np.ones((4))
                re_sig = sig-min_sig
                eb = (phi_a[s]*comparision[t]+phi_b[s]) * re_sig
                pb = np.zeros((4))
                re = (gamma_b[s])*v/np.sum(sig)
                if t>0:
                    if action[t-1]!=4:
                        pb[action[t-1]] = (persev_b[s])
                # elife: beta[s]*(v+eb+pb+re); nhb: (beta[s]*v+eb+pb+re)
                loglikelyhood += action_probability(((beta_b[s])*v+eb+pb+re),action[t])
                pe = reward[t] - v[action[t]]
                Kgain = np.square(sig[action[t]])/(np.square(sig[action[t]])+np.square(sig_o))
                v[action[t]] = v[action[t]] + Kgain * pe
                sig[action[t]] = np.sqrt((1-Kgain)*np.square(sig[action[t]]))
            sig = np.sqrt(np.square(sig)+np.square(sig_d))
        subject_loglikelyhood.append(loglikelyhood)
    return subject_loglikelyhood




def log_likelyhood_glm_absolute(file_name):#nhb
    beta_a,phi_a,persev_a,beta_b,phi_b,persev_b = read_parameter_glm_3_para(file_name)
    subjects = 60
    trials = 150
    subject_loglikelyhood = []
    for s in range(subjects):
        action,reward,reward_B,_ =read_data(s) 
        comparision = rewards2comparison_sqrt(action,reward,reward_B,0)
        loglikelyhood = 0
        sig_o = 4
        sig_d = 2.8
        v = np.ones((4))*50
        sig = np.ones((4))*4
        for t in range(trials):
            if action[t]!=4:
                #beta:temperature; v:action value; eb:exploration bonus; pb: perseverance bonus; re: random exploration
                min_sig = np.min(sig)*np.ones((4))
                re_sig = sig-min_sig
                eb = (phi_a[s]*np.abs(comparision[t])+phi_b[s]) * re_sig
                pb = np.zeros((4))
                if t>0:
                    if action[t-1]!=4:
                        pb[action[t-1]] = (persev_a[s]*np.abs(comparision[t])+persev_b[s])
                # elife: beta[s]*(v+eb+pb+re); nhb: (beta[s]*v+eb+pb+re)
                beta_t = beta_a[s]*np.abs(comparision[t])+beta_b[s]
                if beta_t < 0:
                    beta_t = 0
                loglikelyhood += action_probability((beta_t*v+eb+pb),action[t])
                pe = reward[t] - v[action[t]]
                Kgain = np.square(sig[action[t]])/(np.square(sig[action[t]])+np.square(sig_o))
                v[action[t]] = v[action[t]] + Kgain * pe
                sig[action[t]] = np.sqrt((1-Kgain)*np.square(sig[action[t]]))
            sig = np.sqrt(np.square(sig)+np.square(sig_d))
        subject_loglikelyhood.append(loglikelyhood)
    return subject_loglikelyhood


def log_likelyhood_nhb_glm_ru_no_gamma_sqrt(file_name):#nhb
    beta_a,phi_a,persev_a,beta_b,phi_b,persev_b = read_parameter_glm_3_para(file_name)
    subjects = 60
    trials = 150
    subject_loglikelyhood = []
    for s in range(subjects):
        action,reward,reward_B,_ =read_data(s) 
        comparision = rewards2comparison_sqrt(action,reward,reward_B,0)
        loglikelyhood = 0
        sig_o = 4
        sig_d = 2.8
        v = np.ones((4))*50
        sig = np.ones((4))*4
        for t in range(trials):
            if action[t]!=4:
                #beta:temperature; v:action value; eb:exploration bonus; pb: perseverance bonus; re: random exploration
                min_sig = np.min(sig)*np.ones((4))
                re_sig = sig-min_sig
                eb = (phi_a[s]*comparision[t]+phi_b[s]) * re_sig
                pb = np.zeros((4))
                if t>0:
                    if action[t-1]!=4:
                        pb[action[t-1]] = (persev_a[s]*comparision[t]+persev_b[s])
                # elife: beta[s]*(v+eb+pb+re); nhb: (beta[s]*v+eb+pb+re)
                beta_t = beta_a[s]*comparision[t]+beta_b[s]
                if beta_t < 0:
                    beta_t = 0
                loglikelyhood += action_probability((beta_t*v+eb+pb),action[t])
                pe = reward[t] - v[action[t]]
                Kgain = np.square(sig[action[t]])/(np.square(sig[action[t]])+np.square(sig_o))
                v[action[t]] = v[action[t]] + Kgain * pe
                sig[action[t]] = np.sqrt((1-Kgain)*np.square(sig[action[t]]))
            sig = np.sqrt(np.square(sig)+np.square(sig_d))
        subject_loglikelyhood.append(loglikelyhood)
    return subject_loglikelyhood

def log_likelyhood_nhb_competition_action(file_name):#nhb
    beta_b,phi_b,persev_b,level = read_parameter_competition(file_name)
    subjects = 60
    trials = 150
    subject_loglikelyhood = []
    for s in range(subjects):
        action,reward,reward_B,_ =read_data(s) 
        comparision = rewards2comparison_sqrt(action,reward,reward_B,0)
        loglikelyhood = 0
        sig_o = 4
        sig_d = 2.8
        v = np.ones((4))*50
        sig = np.ones((4))*4
        for t in range(trials):
            if action[t]!=4:
                #beta:temperature; v:action value; eb:exploration bonus; pb: perseverance bonus; re: random exploration
                min_sig = np.min(sig)*np.ones((4))
                re_sig = sig-min_sig
                eb = (phi_b[s]) * re_sig
                pb = np.zeros((4))
                compare = np.zeros((4))
                if t>0:
                    if action[t-1]!=4:
                        pb[action[t-1]] = (persev_b[s])
                        compare[action[t-1]] = comparision[t]
                # elife: beta[s]*(v+eb+pb+re); nhb: (beta[s]*v+eb+pb+re)
                
                happy = v + level[s] * compare
                
                loglikelyhood += action_probability(((beta_b[s])*happy+eb+pb),action[t])
                pe = reward[t] - v[action[t]]
                Kgain = np.square(sig[action[t]])/(np.square(sig[action[t]])+np.square(sig_o))
                v[action[t]] = v[action[t]] + Kgain * pe
                sig[action[t]] = np.sqrt((1-Kgain)*np.square(sig[action[t]]))
            sig = np.sqrt(np.square(sig)+np.square(sig_d))
        subject_loglikelyhood.append(loglikelyhood)
    return subject_loglikelyhood

def log_likelyhood_nhb_competition_learn(file_name):#nhb
    beta_b,phi_b,persev_b,level = read_parameter_competition(file_name)
    subjects = 60
    trials = 150
    subject_loglikelyhood = []
    for s in range(subjects):
        action,reward,reward_B,_ =read_data(s) 
        comparision = rewards2comparison_sqrt_competition_learn(action,reward,reward_B,0)
        loglikelyhood = 0
        sig_o = 4
        sig_d = 2.8
        v = np.ones((4))*50
        sig = np.ones((4))*4
        for t in range(trials):
            if action[t]!=4:
                #beta:temperature; v:action value; eb:exploration bonus; pb: perseverance bonus; re: random exploration
                min_sig = np.min(sig)*np.ones((4))
                re_sig = sig-min_sig
                eb = (phi_b[s]) * re_sig
                pb = np.zeros((4))
                if t>0:
                    if action[t-1]!=4:
                        pb[action[t-1]] = (persev_b[s])
                # elife: beta[s]*(v+eb+pb+re); nhb: (beta[s]*v+eb+pb+re)
                
                loglikelyhood += action_probability(((beta_b[s])*v+eb+pb),action[t])
                pe = reward[t] + level[s]*comparision[t] - v[action[t]]
                Kgain = np.square(sig[action[t]])/(np.square(sig[action[t]])+np.square(sig_o))
                v[action[t]] = v[action[t]] + Kgain * pe
                sig[action[t]] = np.sqrt((1-Kgain)*np.square(sig[action[t]]))
            sig = np.sqrt(np.square(sig)+np.square(sig_d))
        subject_loglikelyhood.append(loglikelyhood)
    return subject_loglikelyhood

def log_likelyhood_nhb_glm2_ru_no_gamma_sqrt(file_name):#nhb
    beta_a_1,phi_a_1,persev_a_1,beta_a_0,phi_a_0,persev_a_0,beta_b,phi_b,persev_b = read_parameter_glm2_no_gamma(file_name)
    subjects = 60
    trials = 150
    subject_loglikelyhood = []
    for s in range(subjects):
        action,reward,reward_B,_ =read_data(s) 
        comparision = rewards2comparison_sqrt(action,reward,reward_B,0)
        loglikelyhood = 0
        sig_o = 4
        sig_d = 2.8
        v = np.ones((4))*50
        sig = np.ones((4))*4
        for t in range(trials):
            if action[t]!=4:
                if comparision[t]<0:
                    
                    #beta:temperature; v:action value; eb:exploration bonus; pb: perseverance bonus; re: random exploration
                    min_sig = np.min(sig)*np.ones((4))
                    re_sig = sig-min_sig
                    eb = (phi_a_0[s]*comparision[t]+phi_b[s]) * re_sig
                    pb = np.zeros((4))
                    if t>0:
                        if action[t-1]!=4:
                            pb[action[t-1]] = (persev_a_0[s]*comparision[t]+persev_b[s])
                    # elife: beta[s]*(v+eb+pb+re); nhb: (beta[s]*v+eb+pb+re)
                    beta_t = beta_a_0[s]*comparision[t]+beta_b[s]
                    if beta_t < 0:
                        beta_t = 0
                    loglikelyhood += action_probability((beta_t*v+eb+pb),action[t])
                if comparision[t]>=0:
                    min_sig = np.min(sig)*np.ones((4))
                    re_sig = sig-min_sig
                    eb = (phi_a_1[s]*comparision[t]+phi_b[s]) * re_sig
                    pb = np.zeros((4))
                    if t>0:
                        if action[t-1]!=4:
                            pb[action[t-1]] = (persev_a_1[s]*comparision[t]+persev_b[s])
                    # elife: beta[s]*(v+eb+pb+re); nhb: (beta[s]*v+eb+pb+re)
                    beta_t = beta_a_1[s]*comparision[t]+beta_b[s]
                    if beta_t < 0:
                        beta_t = 0
                    loglikelyhood += action_probability((beta_t*v+eb+pb),action[t])
            
                pe = reward[t] - v[action[t]]
                Kgain = np.square(sig[action[t]])/(np.square(sig[action[t]])+np.square(sig_o))
                v[action[t]] = v[action[t]] + Kgain * pe
                sig[action[t]] = np.sqrt((1-Kgain)*np.square(sig[action[t]]))
            sig = np.sqrt(np.square(sig)+np.square(sig_d))
        subject_loglikelyhood.append(loglikelyhood)
    return subject_loglikelyhood




def log_likelyhood_nhb_glm2_ru_no_gamma_sqrt_beta(file_name):#nhb
    beta_b,phi_b,persev_b,beta_a_0,beta_a_1 = read_parameter_glm2_no_gamma_beta(file_name)
    subjects = 60
    trials = 150
    subject_loglikelyhood = []
    for s in range(subjects):
        action,reward,reward_B,_ =read_data(s) 
        comparision = rewards2comparison_sqrt(action,reward,reward_B,0)
        loglikelyhood = 0
        sig_o = 4
        sig_d = 2.8
        v = np.ones((4))*50
        sig = np.ones((4))*4
        for t in range(trials):
            if action[t]!=4:
                if comparision[t]<0:
                    
                    #beta:temperature; v:action value; eb:exploration bonus; pb: perseverance bonus; re: random exploration
                    min_sig = np.min(sig)*np.ones((4))
                    re_sig = sig-min_sig
                    eb = (phi_b[s]) * re_sig
                    pb = np.zeros((4))
                    if t>0:
                        if action[t-1]!=4:
                            pb[action[t-1]] = (persev_b[s])
                    # elife: beta[s]*(v+eb+pb+re); nhb: (beta[s]*v+eb+pb+re)
                    beta_t = beta_a_0[s]*comparision[t]+beta_b[s]
                    if beta_t < 0:
                        beta_t = 0
                    loglikelyhood += action_probability((beta_t*v+eb+pb),action[t])
                if comparision[t]>=0:
                    min_sig = np.min(sig)*np.ones((4))
                    re_sig = sig-min_sig
                    eb = (phi_b[s]) * re_sig
                    pb = np.zeros((4))
                    if t>0:
                        if action[t-1]!=4:
                            pb[action[t-1]] = (persev_b[s])
                    # elife: beta[s]*(v+eb+pb+re); nhb: (beta[s]*v+eb+pb+re)
                    beta_t = beta_a_1[s]*comparision[t]+beta_b[s]
                    if beta_t < 0:
                        beta_t = 0
                    loglikelyhood += action_probability((beta_t*v+eb+pb),action[t])
            
                pe = reward[t] - v[action[t]]
                Kgain = np.square(sig[action[t]])/(np.square(sig[action[t]])+np.square(sig_o))
                v[action[t]] = v[action[t]] + Kgain * pe
                sig[action[t]] = np.sqrt((1-Kgain)*np.square(sig[action[t]]))
            sig = np.sqrt(np.square(sig)+np.square(sig_d))
        subject_loglikelyhood.append(loglikelyhood)
    return subject_loglikelyhood

def log_likelyhood_nhb_glm2_ru_no_gamma_sqrt_phi(file_name):#nhb
    beta_b,phi_b,persev_b,phi_a_0,phi_a_1 = read_parameter_glm2_no_gamma_beta(file_name)
    subjects = 60
    trials = 150
    subject_loglikelyhood = []
    for s in range(subjects):
        action,reward,reward_B,_ =read_data(s) 
        comparision = rewards2comparison_sqrt(action,reward,reward_B,0)
        loglikelyhood = 0
        sig_o = 4
        sig_d = 2.8
        v = np.ones((4))*50
        sig = np.ones((4))*4
        for t in range(trials):
            if action[t]!=4:
                if comparision[t]<0:
                    
                    #beta:temperature; v:action value; eb:exploration bonus; pb: perseverance bonus; re: random exploration
                    min_sig = np.min(sig)*np.ones((4))
                    re_sig = sig-min_sig
                    eb = (phi_a_0[s]*comparision[t]+phi_b[s]) * re_sig
                    pb = np.zeros((4))
                    if t>0:
                        if action[t-1]!=4:
                            pb[action[t-1]] = (persev_b[s])
                    # elife: beta[s]*(v+eb+pb+re); nhb: (beta[s]*v+eb+pb+re)
                    loglikelyhood += action_probability(((beta_b[s])*v+eb+pb),action[t])
                if comparision[t]>=0:
                    min_sig = np.min(sig)*np.ones((4))
                    re_sig = sig-min_sig
                    eb = (phi_a_1[s]*comparision[t]+phi_b[s]) * re_sig
                    pb = np.zeros((4))
                    if t>0:
                        if action[t-1]!=4:
                            pb[action[t-1]] = (persev_b[s])
                    # elife: beta[s]*(v+eb+pb+re); nhb: (beta[s]*v+eb+pb+re)
                    loglikelyhood += action_probability(((beta_b[s])*v+eb+pb),action[t])
            
                pe = reward[t] - v[action[t]]
                Kgain = np.square(sig[action[t]])/(np.square(sig[action[t]])+np.square(sig_o))
                v[action[t]] = v[action[t]] + Kgain * pe
                sig[action[t]] = np.sqrt((1-Kgain)*np.square(sig[action[t]]))
            sig = np.sqrt(np.square(sig)+np.square(sig_d))
        subject_loglikelyhood.append(loglikelyhood)
    return subject_loglikelyhood

def log_likelyhood_nhb_glm2_ru_no_gamma_sqrt_persev(file_name):#nhb
    beta_b,phi_b,persev_b,persev_a_0,persev_a_1 = read_parameter_glm2_no_gamma_beta(file_name)
    subjects = 60
    trials = 150
    subject_loglikelyhood = []
    for s in range(subjects):
        action,reward,reward_B,_ =read_data(s) 
        comparision = rewards2comparison_sqrt(action,reward,reward_B,0)
        loglikelyhood = 0
        sig_o = 4
        sig_d = 2.8
        v = np.ones((4))*50
        sig = np.ones((4))*4
        for t in range(trials):
            if action[t]!=4:
                if comparision[t]<0:
                    
                    #beta:temperature; v:action value; eb:exploration bonus; pb: perseverance bonus; re: random exploration
                    min_sig = np.min(sig)*np.ones((4))
                    re_sig = sig-min_sig
                    eb = (phi_b[s]) * re_sig
                    pb = np.zeros((4))
                    if t>0:
                        if action[t-1]!=4:
                            pb[action[t-1]] = (persev_a_0[s]*comparision[t]+persev_b[s])
                    # elife: beta[s]*(v+eb+pb+re); nhb: (beta[s]*v+eb+pb+re)
                    loglikelyhood += action_probability(((beta_b[s])*v+eb+pb),action[t])
                if comparision[t]>=0:
                    min_sig = np.min(sig)*np.ones((4))
                    re_sig = sig-min_sig
                    eb = (phi_b[s]) * re_sig
                    pb = np.zeros((4))
                    if t>0:
                        if action[t-1]!=4:
                            pb[action[t-1]] = (persev_a_1[s]*comparision[t]+persev_b[s])
                    # elife: beta[s]*(v+eb+pb+re); nhb: (beta[s]*v+eb+pb+re)
                    loglikelyhood += action_probability(((beta_b[s])*v+eb+pb),action[t])
            
                pe = reward[t] - v[action[t]]
                Kgain = np.square(sig[action[t]])/(np.square(sig[action[t]])+np.square(sig_o))
                v[action[t]] = v[action[t]] + Kgain * pe
                sig[action[t]] = np.sqrt((1-Kgain)*np.square(sig[action[t]]))
            sig = np.sqrt(np.square(sig)+np.square(sig_d))
        subject_loglikelyhood.append(loglikelyhood)
    return subject_loglikelyhood










def log_likelyhood_nhb_ru_no_gamma(file_name):#nhb
    beta_b,phi_b,persev_b = read_parameter_no_gamma(file_name)
    subjects = 60
    trials = 150
    subject_loglikelyhood = []
    for s in range(subjects):
        action,reward,reward_B,_ =read_data(s) 
        comparision = rewards2comparison_sqrt(action,reward,reward_B,0)
        loglikelyhood = 0
        sig_o = 4
        sig_d = 2.8
        v = np.ones((4))*50
        sig = np.ones((4))*4
        for t in range(trials):
            if action[t]!=4:
                #beta:temperature; v:action value; eb:exploration bonus; pb: perseverance bonus; re: random exploration
                min_sig = np.min(sig)*np.ones((4))
                re_sig = sig-min_sig
                eb = phi_b[s] * re_sig
                pb = np.zeros((4))
                if t>0:
                    if action[t-1]!=4:
                        pb[action[t-1]] = persev_b[s]
                # elife: beta[s]*(v+eb+pb+re); nhb: (beta[s]*v+eb+pb+re)
                loglikelyhood += action_probability((beta_b[s]*v+eb+pb),action[t])
                pe = reward[t] - v[action[t]]
                Kgain = np.square(sig[action[t]])/(np.square(sig[action[t]])+np.square(sig_o))
                v[action[t]] = v[action[t]] + Kgain * pe
                sig[action[t]] = np.sqrt((1-Kgain)*np.square(sig[action[t]]))
            sig = np.sqrt(np.square(sig)+np.square(sig_d))
        subject_loglikelyhood.append(loglikelyhood)
    return subject_loglikelyhood

def log_likelyhood_nhb_ru_no_gamma_no_persev(file_name):#nhb
    beta_b,phi_b = read_parameter_no_gamma_no_persev(file_name)
    subjects = 60
    trials = 150
    subject_loglikelyhood = []
    for s in range(subjects):
        action,reward,reward_B,_ =read_data(s) 
        comparision = rewards2comparison_sqrt(action,reward,reward_B,0)
        loglikelyhood = 0
        sig_o = 4
        sig_d = 2.8
        v = np.ones((4))*50
        sig = np.ones((4))*4
        for t in range(trials):
            if action[t]!=4:
                #beta:temperature; v:action value; eb:exploration bonus; pb: perseverance bonus; re: random exploration
                min_sig = np.min(sig)*np.ones((4))
                re_sig = sig-min_sig
                eb = phi_b[s] * re_sig
                # elife: beta[s]*(v+eb+pb+re); nhb: (beta[s]*v+eb+pb+re)
                loglikelyhood += action_probability((beta_b[s]*v+eb),action[t])
                pe = reward[t] - v[action[t]]
                Kgain = np.square(sig[action[t]])/(np.square(sig[action[t]])+np.square(sig_o))
                v[action[t]] = v[action[t]] + Kgain * pe
                sig[action[t]] = np.sqrt((1-Kgain)*np.square(sig[action[t]]))
            sig = np.sqrt(np.square(sig)+np.square(sig_d))
        subject_loglikelyhood.append(loglikelyhood)
    return subject_loglikelyhood

def log_likelyhood_nhb_ru_no_gamma_no_persev_no_phi(file_name):#nhb
    beta_b = read_parameter_no_gamma_no_persev_no_phi(file_name)
    subjects = 60
    trials = 150
    subject_loglikelyhood = []
    for s in range(subjects):
        action,reward,reward_B,_ =read_data(s) 
        comparision = rewards2comparison_sqrt(action,reward,reward_B,0)
        loglikelyhood = 0
        sig_o = 4
        sig_d = 2.8
        v = np.ones((4))*50
        sig = np.ones((4))*4
        for t in range(trials):
            if action[t]!=4:
                #beta:temperature; v:action value; eb:exploration bonus; pb: perseverance bonus; re: random exploration
                # elife: beta[s]*(v+eb+pb+re); nhb: (beta[s]*v+eb+pb+re)
                loglikelyhood += action_probability((beta_b[s]*v),action[t])
                pe = reward[t] - v[action[t]]
                Kgain = np.square(sig[action[t]])/(np.square(sig[action[t]])+np.square(sig_o))
                v[action[t]] = v[action[t]] + Kgain * pe
                sig[action[t]] = np.sqrt((1-Kgain)*np.square(sig[action[t]]))
            sig = np.sqrt(np.square(sig)+np.square(sig_d))
        subject_loglikelyhood.append(loglikelyhood)
    return subject_loglikelyhood

def log_likelyhood_sup_delta(file_name):#nhb
    beta_b,phi_b,persev_b,alpha = read_parameter_sup(file_name)
    subjects = 60
    trials = 150
    subject_loglikelyhood = []
    for s in range(subjects):
        action,reward,reward_B,_ =read_data(s) 
        comparision = rewards2comparison_sqrt(action,reward,reward_B,0)
        loglikelyhood = 0
        sig_o = 4
        sig_d = 2.8
        v = np.ones((4))*50
        sig = np.ones((4))*4
        for t in range(trials):
            if action[t]!=4:
                #beta:temperature; v:action value; eb:exploration bonus; pb: perseverance bonus; re: random exploration
                min_sig = np.min(sig)*np.ones((4))
                re_sig = sig-min_sig
                eb = phi_b[s] * re_sig
                pb = np.zeros((4))
                if t>0:
                    if action[t-1]!=4:
                        pb[action[t-1]] = persev_b[s]
                # elife: beta[s]*(v+eb+pb+re); nhb: (beta[s]*v+eb+pb+re)
                loglikelyhood += action_probability((beta_b[s]*v+eb+pb),action[t])
                pe = reward[t] - v[action[t]]
                Kgain = np.square(sig[action[t]])/(np.square(sig[action[t]])+np.square(sig_o))
                v[action[t]] = v[action[t]] + alpha[s] * pe
                sig[action[t]] = np.sqrt((1-Kgain)*np.square(sig[action[t]]))
            sig = np.sqrt(np.square(sig)+np.square(sig_d))
        subject_loglikelyhood.append(loglikelyhood)
    return subject_loglikelyhood

def log_likelyhood_sup_dual_delta(file_name):#nhb
    beta_b,phi_b,persev_b,pos_alpha,neg_alpha = read_parameter_sup_dual(file_name)
    subjects = 60
    trials = 150
    subject_loglikelyhood = []
    for s in range(subjects):
        action,reward,reward_B,_ =read_data(s) 
        comparision = rewards2comparison_sqrt(action,reward,reward_B,0)
        loglikelyhood = 0
        sig_o = 4
        sig_d = 2.8
        v = np.ones((4))*50
        sig = np.ones((4))*4
        for t in range(trials):
            if action[t]!=4:
                #beta:temperature; v:action value; eb:exploration bonus; pb: perseverance bonus; re: random exploration
                min_sig = np.min(sig)*np.ones((4))
                re_sig = sig-min_sig
                eb = phi_b[s] * re_sig
                pb = np.zeros((4))
                if t>0:
                    if action[t-1]!=4:
                        pb[action[t-1]] = persev_b[s]
                # elife: beta[s]*(v+eb+pb+re); nhb: (beta[s]*v+eb+pb+re)
                loglikelyhood += action_probability((beta_b[s]*v+eb+pb),action[t])
                pe = reward[t] - v[action[t]]
                Kgain = np.square(sig[action[t]])/(np.square(sig[action[t]])+np.square(sig_o))
                if pe>0:
                    v[action[t]] = v[action[t]] + pos_alpha[s] * pe
                else:
                    v[action[t]] = v[action[t]] + neg_alpha[s] * pe
                sig[action[t]] = np.sqrt((1-Kgain)*np.square(sig[action[t]]))
            sig = np.sqrt(np.square(sig)+np.square(sig_d))
        subject_loglikelyhood.append(loglikelyhood)
    return subject_loglikelyhood
def log_likelyhood_sup_elife(file_name):#nhb
    beta_b,phi_b,persev_b = read_parameter_no_gamma(file_name)
    subjects = 60
    trials = 150
    subject_loglikelyhood = []
    for s in range(subjects):
        action,reward,reward_B,_ =read_data(s) 
        comparision = rewards2comparison_sqrt(action,reward,reward_B,0)
        loglikelyhood = 0
        sig_o = 4
        sig_d = 2.8
        v = np.ones((4))*50
        sig = np.ones((4))*4
        for t in range(trials):
            if action[t]!=4:
                #beta:temperature; v:action value; eb:exploration bonus; pb: perseverance bonus; re: random exploration
                min_sig = np.min(sig)*np.ones((4))
                re_sig = sig-min_sig
                eb = phi_b[s] * re_sig
                pb = np.zeros((4))
                if t>0:
                    if action[t-1]!=4:
                        pb[action[t-1]] = persev_b[s]
                # elife: beta[s]*(v+eb+pb+re); nhb: (beta[s]*v+eb+pb+re)
                loglikelyhood += action_probability(beta_b[s]*(v+eb+pb),action[t])
                pe = reward[t] - v[action[t]]
                Kgain = np.square(sig[action[t]])/(np.square(sig[action[t]])+np.square(sig_o))
                v[action[t]] = v[action[t]] + Kgain * pe
                sig[action[t]] = np.sqrt((1-Kgain)*np.square(sig[action[t]]))
            sig = np.sqrt(np.square(sig)+np.square(sig_d))
        subject_loglikelyhood.append(loglikelyhood)
    return subject_loglikelyhood
def log_likelyhood_sup_gamma(file_name):#nhb
    beta_b,phi_b,persev_b,gamma_b = read_parameter_sup(file_name)
    subjects = 60
    trials = 150
    subject_loglikelyhood = []
    for s in range(subjects):
        action,reward,reward_B,_ =read_data(s) 
        comparision = rewards2comparison_sqrt(action,reward,reward_B,0)
        loglikelyhood = 0
        sig_o = 4
        sig_d = 2.8
        v = np.ones((4))*50
        sig = np.ones((4))*4
        for t in range(trials):
            if action[t]!=4:
                #beta:temperature; v:action value; eb:exploration bonus; pb: perseverance bonus; re: random exploration
                min_sig = np.min(sig)*np.ones((4))
                re_sig = sig-min_sig
                eb = phi_b[s] * re_sig
                pb = np.zeros((4))
                re = gamma_b[s] * v / np.sum(sig)
                if t>0:
                    if action[t-1]!=4:
                        pb[action[t-1]] = persev_b[s]
                # elife: beta[s]*(v+eb+pb+re); nhb: (beta[s]*v+eb+pb+re)
                loglikelyhood += action_probability((beta_b[s]*v+eb+pb+re),action[t])
                pe = reward[t] - v[action[t]]
                Kgain = np.square(sig[action[t]])/(np.square(sig[action[t]])+np.square(sig_o))
                v[action[t]] = v[action[t]] + Kgain * pe
                sig[action[t]] = np.sqrt((1-Kgain)*np.square(sig[action[t]]))
            sig = np.sqrt(np.square(sig)+np.square(sig_d))
        subject_loglikelyhood.append(loglikelyhood)
    return subject_loglikelyhood

def log_likelyhood_persev_trace(file_name):#nhb
    beta_a,phi_a,persev_a,beta_b,phi_b,persev_b,tau = read_parameter_persev_trace(file_name)
    subjects = 60
    trials = 150
    subject_loglikelyhood = []
    for s in range(subjects):
        action,reward,reward_B,_ =read_data(s) 
        comparision = rewards2comparison_sqrt(action,reward,reward_B,0)
        loglikelyhood = 0
        sig_o = 4
        sig_d = 2.8
        v = np.ones((4))*50
        sig = np.ones((4))*4
        pb_last = np.zeros((4))
        for t in range(trials):
            if action[t]!=4:
                #beta:temperature; v:action value; eb:exploration bonus; pb: perseverance bonus; re: random exploration
                min_sig = np.min(sig)*np.ones((4))
                re_sig = sig-min_sig
                eb = (phi_a[s]*comparision[t]+phi_b[s]) * re_sig
                pb = np.zeros((4))
                if t>0:
                    if action[t-1]!=4:
                        pb[action[t-1]] = (persev_a[s]*comparision[t]+persev_b[s])
                pb = pb + tau[s]*pb_last
                pb_last = pb
                # elife: beta[s]*(v+eb+pb+re); nhb: (beta[s]*v+eb+pb+re)
                beta_t = beta_a[s]*comparision[t]+beta_b[s]
                if beta_t < 0:
                    beta_t = 0
                loglikelyhood += action_probability((beta_t*v+eb+pb),action[t])
                pe = reward[t] - v[action[t]]
                Kgain = np.square(sig[action[t]])/(np.square(sig[action[t]])+np.square(sig_o))
                v[action[t]] = v[action[t]] + Kgain * pe
                sig[action[t]] = np.sqrt((1-Kgain)*np.square(sig[action[t]]))
            sig = np.sqrt(np.square(sig)+np.square(sig_d))
        subject_loglikelyhood.append(loglikelyhood)
    return subject_loglikelyhood

def log_likelyhood_persev_trace_glm_p(file_name):#nhb
    persev_a,beta_b,phi_b,persev_b,tau = read_parameter_persev_trace_glm_p(file_name)
    subjects = 60
    trials = 150
    subject_loglikelyhood = []
    for s in range(subjects):
        action,reward,reward_B,_ =read_data(s) 
        comparision = rewards2comparison_sqrt(action,reward,reward_B,0)
        loglikelyhood = 0
        sig_o = 4
        sig_d = 2.8
        v = np.ones((4))*50
        sig = np.ones((4))*4
        pb_last = np.zeros((4))
        for t in range(trials):
            if action[t]!=4:
                #beta:temperature; v:action value; eb:exploration bonus; pb: perseverance bonus; re: random exploration
                min_sig = np.min(sig)*np.ones((4))
                re_sig = sig-min_sig
                eb = phi_b[s] * re_sig
                pb = np.zeros((4))
                if t>0:
                    if action[t-1]!=4:
                        pb[action[t-1]] = (persev_a[s]*comparision[t]+persev_b[s])
                pb = pb + tau[s]*pb_last
                pb_last = pb
                # elife: beta[s]*(v+eb+pb+re); nhb: (beta[s]*v+eb+pb+re)
                loglikelyhood += action_probability((beta_b[s]*v+eb+pb),action[t])
                pe = reward[t] - v[action[t]]
                Kgain = np.square(sig[action[t]])/(np.square(sig[action[t]])+np.square(sig_o))
                v[action[t]] = v[action[t]] + Kgain * pe
                sig[action[t]] = np.sqrt((1-Kgain)*np.square(sig[action[t]]))
            sig = np.sqrt(np.square(sig)+np.square(sig_d))
        subject_loglikelyhood.append(loglikelyhood)
    return subject_loglikelyhood

def log_likelyhood_persev_trace_vep(file_name):#nhb
    beta_b,phi_b,persev_b,tau = read_parameter_persev_trace_vep(file_name)
    subjects = 60
    trials = 150
    subject_loglikelyhood = []
    for s in range(subjects):
        action,reward,reward_B,_ =read_data(s) 
        comparision = rewards2comparison_sqrt(action,reward,reward_B,0)
        loglikelyhood = 0
        sig_o = 4
        sig_d = 2.8
        v = np.ones((4))*50
        sig = np.ones((4))*4
        pb_last = np.zeros((4))
        for t in range(trials):
            if action[t]!=4:
                #beta:temperature; v:action value; eb:exploration bonus; pb: perseverance bonus; re: random exploration
                min_sig = np.min(sig)*np.ones((4))
                re_sig = sig-min_sig
                eb = phi_b[s] * re_sig
                pb = np.zeros((4))
                if t>0:
                    if action[t-1]!=4:
                        pb[action[t-1]] = persev_b[s]
                pb = pb + tau[s]*pb_last
                pb_last = pb
                # elife: beta[s]*(v+eb+pb+re); nhb: (beta[s]*v+eb+pb+re)
                loglikelyhood += action_probability((beta_b[s]*v+eb+pb),action[t])
                pe = reward[t] - v[action[t]]
                Kgain = np.square(sig[action[t]])/(np.square(sig[action[t]])+np.square(sig_o))
                v[action[t]] = v[action[t]] + Kgain * pe
                sig[action[t]] = np.sqrt((1-Kgain)*np.square(sig[action[t]]))
            sig = np.sqrt(np.square(sig)+np.square(sig_d))
        subject_loglikelyhood.append(loglikelyhood)
    return subject_loglikelyhood

def log_likelyhood_social_comparison_trace(file_name):#nhb
    beta_a,phi_a,persev_a,beta_b,phi_b,persev_b = read_parameter_glm_3_para(file_name)
    subjects = 60
    trials = 150
    subject_loglikelyhood = []
    for s in range(subjects):
        action,reward,reward_B,_ =read_data(s) 
        comparision = rewards2comparison_sqrt(action,reward,reward_B,0.01)
        loglikelyhood = 0
        sig_o = 4
        sig_d = 2.8
        v = np.ones((4))*50
        sig = np.ones((4))*4
        for t in range(trials):
            if action[t]!=4:
                #beta:temperature; v:action value; eb:exploration bonus; pb: perseverance bonus; re: random exploration
                min_sig = np.min(sig)*np.ones((4))
                re_sig = sig-min_sig
                eb = (phi_a[s]*comparision[t]+phi_b[s]) * re_sig
                pb = np.zeros((4))
                if t>0:
                    if action[t-1]!=4:
                        pb[action[t-1]] = (persev_a[s]*comparision[t]+persev_b[s])
                # elife: beta[s]*(v+eb+pb+re); nhb: (beta[s]*v+eb+pb+re)
                beta_t = beta_a[s]*comparision[t]+beta_b[s]
                if beta_t < 0:
                    beta_t = 0
                loglikelyhood += action_probability((beta_t*v+eb+pb),action[t])
                pe = reward[t] - v[action[t]]
                Kgain = np.square(sig[action[t]])/(np.square(sig[action[t]])+np.square(sig_o))
                v[action[t]] = v[action[t]] + Kgain * pe
                sig[action[t]] = np.sqrt((1-Kgain)*np.square(sig[action[t]]))
            sig = np.sqrt(np.square(sig)+np.square(sig_d))
        subject_loglikelyhood.append(loglikelyhood)
    return subject_loglikelyhood

def log_likelyhood_social_comparison_trace_tau(file_name):#nhb
    beta_a,phi_a,persev_a,beta_b,phi_b,persev_b,tau = read_parameter_glm_tau(file_name)
    subjects = 60
    trials = 150
    subject_loglikelyhood = []
    for s in range(subjects):
        action,reward,reward_B,_ =read_data(s) 
        comparision = rewards2comparison(action,reward,reward_B,0.00)
        loglikelyhood = 0
        sig_o = 4
        sig_d = 2.8
        v = np.ones((4))*50
        sig = np.ones((4))*4
        com = 0
        for t in range(trials):
            if action[t]!=4:
                com = tau[s]*com + comparision[t]
                #beta:temperature; v:action value; eb:exploration bonus; pb: perseverance bonus; re: random exploration
                min_sig = np.min(sig)*np.ones((4))
                re_sig = sig-min_sig
                eb = (phi_a[s]*com+phi_b[s]) * re_sig
                pb = np.zeros((4))
                if t>0:
                    if action[t-1]!=4:
                        pb[action[t-1]] = (persev_a[s]*com+persev_b[s])
                # elife: beta[s]*(v+eb+pb+re); nhb: (beta[s]*v+eb+pb+re)
                beta_t = beta_a[s]*com+beta_b[s]
                if beta_t < 0:
                    beta_t = 0
                loglikelyhood += action_probability((beta_t*v+eb+pb),action[t])
                pe = reward[t] - v[action[t]]
                Kgain = np.square(sig[action[t]])/(np.square(sig[action[t]])+np.square(sig_o))
                v[action[t]] = v[action[t]] + Kgain * pe
                sig[action[t]] = np.sqrt((1-Kgain)*np.square(sig[action[t]]))
            sig = np.sqrt(np.square(sig)+np.square(sig_d))
        subject_loglikelyhood.append(loglikelyhood)
    return subject_loglikelyhood

def log_likelyhood_nhb_glm_elife(file_name):#nhb
    beta_a,phi_a,persev_a,beta_b,phi_b,persev_b = read_parameter_glm_3_para(file_name)
    subjects = 60
    trials = 150
    subject_loglikelyhood = []
    for s in range(subjects):
        action,reward,reward_B,_ =read_data(s) 
        comparision = rewards2comparison_sqrt(action,reward,reward_B,0)
        loglikelyhood = 0
        sig_o = 4
        sig_d = 2.8
        v = np.ones((4))*50
        sig = np.ones((4))*4
        for t in range(trials):
            if action[t]!=4:
                #beta:temperature; v:action value; eb:exploration bonus; pb: perseverance bonus; re: random exploration
                min_sig = np.min(sig)*np.ones((4))
                re_sig = sig-min_sig
                eb = (phi_a[s]*comparision[t]+phi_b[s]) * re_sig
                pb = np.zeros((4))
                if t>0:
                    if action[t-1]!=4:
                        pb[action[t-1]] = (persev_a[s]*comparision[t]+persev_b[s])
                # elife: beta[s]*(v+eb+pb+re); nhb: (beta[s]*v+eb+pb+re)
                beta_t = beta_a[s]*comparision[t]+beta_b[s]
                if beta_t < 0:
                    beta_t = 0
                loglikelyhood += action_probability((beta_t*(v+eb+pb)),action[t])
                pe = reward[t] - v[action[t]]
                Kgain = np.square(sig[action[t]])/(np.square(sig[action[t]])+np.square(sig_o))
                v[action[t]] = v[action[t]] + Kgain * pe
                sig[action[t]] = np.sqrt((1-Kgain)*np.square(sig[action[t]]))
            sig = np.sqrt(np.square(sig)+np.square(sig_d))
        subject_loglikelyhood.append(loglikelyhood)
    return subject_loglikelyhood

def log_likelyhood_nhb_glm_elife_beta_phi(file_name):#nhb
    beta_a,phi_a,beta_b,phi_b,persev_b = read_parameter_nhb_delta(file_name)
    subjects = 60
    trials = 150
    subject_loglikelyhood = []
    for s in range(subjects):
        action,reward,reward_B,_ =read_data(s) 
        comparision = rewards2comparison_sqrt(action,reward,reward_B,0)
        loglikelyhood = 0
        sig_o = 4
        sig_d = 2.8
        v = np.ones((4))*50
        sig = np.ones((4))*4
        for t in range(trials):
            if action[t]!=4:
                #beta:temperature; v:action value; eb:exploration bonus; pb: perseverance bonus; re: random exploration
                min_sig = np.min(sig)*np.ones((4))
                re_sig = sig-min_sig
                eb = (phi_a[s]*comparision[t]+phi_b[s]) * re_sig
                pb = np.zeros((4))
                if t>0:
                    if action[t-1]!=4:
                        pb[action[t-1]] = (persev_b[s])
                # elife: beta[s]*(v+eb+pb+re); nhb: (beta[s]*v+eb+pb+re)
                beta_t = beta_a[s]*comparision[t]+beta_b[s]
                if beta_t < 0:
                    beta_t = 0

                loglikelyhood += action_probability((beta_t*(v+eb+pb)),action[t])
                pe = reward[t] - v[action[t]]
                Kgain = np.square(sig[action[t]])/(np.square(sig[action[t]])+np.square(sig_o))
                v[action[t]] = v[action[t]] + Kgain * pe
                sig[action[t]] = np.sqrt((1-Kgain)*np.square(sig[action[t]]))
            sig = np.sqrt(np.square(sig)+np.square(sig_d))
        subject_loglikelyhood.append(loglikelyhood)
    return subject_loglikelyhood

In [9]:
glm_vep_loo = loo_glm_vep('./fitting_result/model_glm_vep.csv')
print(np.sum(glm_vep_loo))

-9318.045043221282


In [10]:
glm_vep_loo_test = loo_glm_vep('./fitting_result/model_glm_vep_test.csv')
print(np.sum(glm_vep_loo_test))

-9098.31086636879


In [11]:
glm_vep_loo_test2 = loo_glm_vep('./fitting_result/model_glm_vep_test2.csv')
print(np.sum(glm_vep_loo_test2))

-8992.755804438837


In [23]:
glm2_vep_loo = loo_sup_glm2_vep('./fitting_result/model_glm2_vep.csv')
print(np.sum(glm2_vep_loo))

-17181.381979683967


In [24]:
glm_vep_llh = log_likelyhood_nhb_glm_ru_no_gamma_sqrt('./fitting_result/model_glm_vep.csv')
print(np.sum(glm_vep_llh))

-7615.329454758791


In [25]:
glm_vep_llh_test = log_likelyhood_nhb_glm_ru_no_gamma_sqrt('./fitting_result/model_glm_vep_test.csv')
print(np.sum(glm_vep_llh_test))

-7613.280339121779


In [26]:
glm_vep_llh_test2 = log_likelyhood_nhb_glm_ru_no_gamma_sqrt('./fitting_result/model_glm_vep_test2.csv')
print(np.sum(glm_vep_llh_test2))

-7612.593838506918


In [31]:
glm2_vep_llh = log_likelyhood_nhb_glm2_ru_no_gamma_sqrt('./fitting_result/model_glm2_vep.csv')
print(np.sum(glm2_vep_llh))

-7588.5134864233
